<a href="https://colab.research.google.com/github/banghj-kr/python/blob/gScore/gScore_02_Data_ML_Colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 01. 분석을 위한 기초 작업

## 01-1. 라이브러리 로딩 및 함수 정의

In [ ]:
# 01-1. 필요한 라이브러리 로딩 및 함수 정의 (240808)
print("01-1. 필요한 라이브러리 로딩 및 함수 정의")

# 필요한 라이브러리 로딩
import pandas as pd
import numpy as np
from datetime import datetime
import pytz
import os
from IPython import get_ipython

def check_environment():
    try:
        if 'google.colab' in env:  # Colab 환경
            from google.colab import data_table, drive

            # 구글 드라이브 마운트 - Colab 환경 only
            drive.mount('/content/drive')

            # 데이터프레임을 구글 코랩의 데이터 테이블로 출력 활성화 - Colab 환경 only
            data_table.enable_dataframe_formatter()

            # 기본 경로 설정
            base_path = '/content/drive/MyDrive/Colab Notebooks/gScore/'
            return base_path

        elif 'ZMQInteractiveShell' in env:  # Jupyter 환경
            # 기본 경로 설정
            base_path = ''
            return base_path
        else:
            return None
    except ImportError:
        return None

# 서울 시간대 설정
tz = pytz.timezone('Asia/Seoul')

# 환경에 맞는 기본 경로 설정
env = str(get_ipython())  # 실행 환경 확인
base_path = check_environment()
print(f"  - 기본 경로를 '{base_path}'로 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# pandas display 옵션 설정
pd.set_option('display.max_columns', 12)
pd.set_option('display.max_rows', 100)

# 필요한 함수 정의

# 데이터프레임의 정보 출력 함수 정의
def display_info_in_sections(df, step=50):
    for i in range(0, len(df.columns), step):
        print(df.iloc[:, i:i+step].info())

print(f"  - 라이브러리 로딩 및 함수 정의를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

## 01-2. 데이터 로딩 및 전처리

In [ ]:
# 01-2. 데이터 로딩 (240628)
print("01-2. 데이터 로딩")

# 파일 경로 설정
sub_directory_name = 'Data'
input_file_name = 'gScore_Data_Expanded.parquet'
input_file_path = os.path.join(base_path, sub_directory_name, input_file_name)

# Parquet 파일 읽기
raw_df = pd.read_parquet(input_file_path, engine='pyarrow')
print(f"  - '{input_file_path}' 파일을 읽었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Raw-Data DataFrame (raw_df):")
display(raw_df.head())

# 데이터프레임의 정보 출력
print("\n  * Raw-Data DataFrame Information (raw_df):")
display_info_in_sections(raw_df)

# ==================================================

In [ ]:
# 01-3. 필요 없는 변수 제거 (240622)
print("01-3. 필요 없는 변수 제거")

# 제거할 변수 목록 설정
remove_columns = [col for col in raw_df.columns if col.startswith(('sap_', 'st_', 'gir_', 'scrbl_', 'companion_', 'year_no', 'address')) or col.endswith('_total')]

# 변수 제거
analysis_df = raw_df.drop(columns=remove_columns)

# 데이터프레임의 데이터 일부 출력
print("  * Raw-Data DataFrame (analysis_df):")
display(analysis_df.head())

# 데이터프레임의 정보 출력
print("\n  * Raw-Data DataFrame Information (analysis_df):")
display_info_in_sections(analysis_df)

# ==================================================

In [ ]:
# 01-4. 주요 변수 결측치 정리 (240630)
print("01-4. 주요 변수 결측치 정리")

# is_true 열을 True로 초기화
analysis_df['is_true'] = True

# 첫 번째 홀 스트로크 기록이 -1인 경우를 확인하여 'is_true'를 False로 변경
def check_stroke_f9(row):
    return row['stroke_f9_1'] != -1

# is_true 열을 조건에 따라 변경
analysis_df['is_true'] = analysis_df.apply(check_stroke_f9, axis=1)

# is_true가 False인 행의 특정 열 값을 NaN으로 변경
def clear_false_rows(row):
    if not row['is_true']:
        for i in range(1, 10):
            row[f'stroke_f9_{i}'] = np.nan
            row[f'putt_f9_{i}'] = np.nan
            row[f'penalty_f9_{i}'] = np.nan
            row[f'fw_hit_f9_{i}'] = np.nan
            row[f'stroke_b9_{i}'] = np.nan
            row[f'putt_b9_{i}'] = np.nan
            row[f'penalty_b9_{i}'] = np.nan
            row[f'fw_hit_b9_{i}'] = np.nan
    return row

# 적용
analysis_df = analysis_df.apply(clear_false_rows, axis=1)

# 결과 확인
print("  - is_true가 False인 행의 수정 결과:")
print(analysis_df.loc[~analysis_df['is_true'], [f'stroke_f9_{i}' for i in range(1, 10)] +
                      [f'putt_f9_{i}' for i in range(1, 10)] +
                      [f'penalty_f9_{i}' for i in range(1, 10)] +
                      [f'fw_hit_f9_{i}' for i in range(1, 10)] + ['is_true']].head(10))

# 데이터프레임의 데이터 일부 출력
print("\n  * Raw-Data DataFrame (analysis_df):")
display(analysis_df.head(5))
display(analysis_df.tail(5))

# 데이터프레임의 정보 출력
print("\n  * Raw-Data DataFrame Information (analysis_df):")
display_info_in_sections(analysis_df)

# ==================================================

In [ ]:
# 01-5. 라운딩 데이터를 홀 단위로 정리 (240622)
print("01-5. 라운딩 데이터를 홀 단위로 정리")

# 새로운 데이터프레임을 담을 리스트
expanded_data = []

# 각 라운딩 데이터를 홀 단위로 분리
for idx, row in analysis_df.iterrows():
    for i in range(9):
        # 전반 9홀 데이터 추가
        expanded_data.append({
            'date_time': row['date_time'],
            'no': row['no'],
            'golf_course': row['golf_course'],
            'course': row['course_f9'],
            'course_type': 'f9',
            'hole_no': i + 1,
            'score': row[f'score_f9_{i+1}'],
            'stroke': row[f'stroke_f9_{i+1}'] if pd.notna(row[f'stroke_f9_{i+1}']) else None,
            'putt': row[f'putt_f9_{i+1}'] if pd.notna(row[f'putt_f9_{i+1}']) else None,
            'penalty': row[f'penalty_f9_{i+1}'] if pd.notna(row[f'penalty_f9_{i+1}']) else None,
            'par': row[f'par_f9_{i+1}'] if pd.notna(row[f'par_f9_{i+1}']) else None,
            'fw_hit': row[f'fw_hit_f9_{i+1}'] if pd.notna(row[f'fw_hit_f9_{i+1}']) else None,
            'is_true': row['is_true']
        })

        # 후반 9홀 데이터 추가
        expanded_data.append({
            'date_time': row['date_time'],
            'no': row['no'],
            'golf_course': row['golf_course'],
            'course': row['course_b9'],
            'course_type': 'b9',
            'hole_no': i + 1,
            'score': row[f'score_b9_{i+1}'],
            'stroke': row[f'stroke_b9_{i+1}'] if pd.notna(row[f'stroke_b9_{i+1}']) else None,
            'putt': row[f'putt_b9_{i+1}'] if pd.notna(row[f'putt_b9_{i+1}']) else None,
            'penalty': row[f'penalty_b9_{i+1}'] if pd.notna(row[f'penalty_b9_{i+1}']) else None,
            'par': row[f'par_b9_{i+1}'] if pd.notna(row[f'par_b9_{i+1}']) else None,
            'fw_hit': row[f'fw_hit_b9_{i+1}'] if pd.notna(row[f'fw_hit_b9_{i+1}']) else None,
            'is_true': row['is_true']
        })

# 데이터프레임으로 변환
expanded_df = pd.DataFrame(expanded_data)

# 정렬
expanded_df.sort_values(by=['date_time', 'course_type', 'hole_no'], ascending=[True, False, True], inplace=True)

# 인덱스 재설정
expanded_df.reset_index(drop=True, inplace=True)

# 데이터프레임의 데이터 일부 출력
print("\n  * Expanded Data DataFrame (expanded_df):")
display(expanded_df.head(5))
display(expanded_df.tail(5))

# 데이터프레임의 정보 출력
print("\n  * Expanded Data DataFrame Information (expanded_df):")
display_info_in_sections(expanded_df)

# ==================================================

In [ ]:
# 01-6. 머신러닝을 위한 데이터 처리 (240622)
print("01-6. 머신러닝을 위한 데이터 처리")

# date_time 변수를 날짜와 시간 관련 변수로 변환
expanded_df['date_time'] = pd.to_datetime(expanded_df['date_time'])
expanded_df['month'] = expanded_df['date_time'].dt.month
expanded_df['hour'] = expanded_df['date_time'].dt.hour
expanded_df['season'] = expanded_df['date_time'].dt.month % 12 // 3 + 1  # 봄=1, 여름=2, 가을=3, 겨울=4

# 분석에 사용할 변수 목록 설정
features = ['no', 'hole_no', 'par', 'month', 'hour', 'season']

# course_type을 원핫 인코딩하고 하나의 변수를 제거
expanded_df = pd.get_dummies(expanded_df, columns=['course_type'])
expanded_df.drop(columns=['course_type_f9'], inplace=True)  # course_type_f9 변수 제거

# 머신러닝을 위한 데이터프레임 분리
train_df = expanded_df[expanded_df['is_true']].copy()
predict_df = expanded_df[~expanded_df['is_true']].copy()

# 새로운 파생 변수 생성
train_df['score_minus_stroke'] = train_df['score'] - train_df['stroke']
train_df['score_minus_stroke_minus_penalty'] = train_df['score_minus_stroke'] - train_df['penalty']
predict_df['score_minus_stroke'] = predict_df['score'] - predict_df['stroke']
predict_df['score_minus_stroke_minus_penalty'] = predict_df['score_minus_stroke'] - predict_df['penalty']

# stroke 예측을 위한 데이터
X_train_stroke = train_df[features + ['course_type_b9', 'score']]
y_train_stroke = train_df['stroke']

# penalty 예측을 위한 데이터 (score_minus_stroke 추가)
X_train_penalty = train_df[features + ['course_type_b9', 'score', 'stroke', 'score_minus_stroke']]
y_train_penalty = train_df['penalty']

# putt 예측을 위한 데이터 (score_minus_stroke_minus_penalty 추가)
X_train_putt = train_df[features + ['course_type_b9', 'score', 'stroke', 'penalty', 'score_minus_stroke_minus_penalty']]
y_train_putt = train_df['putt']

# fw_hit 예측을 위한 데이터 (score_minus_stroke_minus_penalty 추가)
X_train_fw_hit = train_df[train_df['par'] != 3][features + ['course_type_b9', 'stroke', 'penalty', 'putt']]
y_train_fw_hit = train_df.loc[train_df['par'] != 3, 'fw_hit']

# 예측을 위한 데이터프레임 준비
X_predict_stroke = predict_df[features + ['course_type_b9', 'score']]
X_predict_penalty = predict_df[features + ['course_type_b9', 'score', 'stroke', 'score_minus_stroke']]
X_predict_putt = predict_df[features + ['course_type_b9', 'score', 'stroke', 'penalty', 'score_minus_stroke_minus_penalty']]
X_predict_fw_hit = predict_df[predict_df['par'] != 3][features + ['course_type_b9', 'stroke', 'penalty', 'putt']]

print(f"  - 데이터 처리를 완료하였습니다. 학습 및 예측을 위한 데이터가 준비되었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Training DataFrame (train_df):")
display(train_df.head())

print("\n  * Prediction DataFrame (predict_df):")
display(predict_df.head())

# 데이터프레임의 정보 출력
print("\n  * Training DataFrame Information (train_df):")
display_info_in_sections(train_df)

print("\n  * Prediction DataFrame Information (predict_df):")
display_info_in_sections(predict_df)

# ==================================================

# 02. 머신러닝 모델을 이용한 예측 과정

## 02-1. 라이브러리 로딩 및 함수 정의

### 02-1-1. 머신러닝에 필요한 라이브러리 로딩 및 기본 설정

In [ ]:
# 02-1-1. 머신러닝에 필요한 라이브러리 로딩 및 기본 설정 (240808)
print("02-1-1. 머신러닝에 필요한 라이브러리 로딩 및 기본 설정")
#!pip install shap
#!pip install xgboost
#!pip install lightgbm
#!pip install catboost
# 필요한 라이브러리 로딩
from sklearn.model_selection import train_test_split, KFold, GridSearchCV, ParameterGrid
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score, accuracy_score, precision_score, recall_score, f1_score
import matplotlib.pyplot as plt
import numpy as np
import os
import json
import joblib
import shap
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.svm import SVR, SVC
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostRegressor, CatBoostClassifier
from sklearn.neighbors import KNeighborsRegressor, KNeighborsClassifier

# 모델 저장 기본 위치 설정
model_sub_directory_name = 'Model'
model_file_path = os.path.join(base_path, model_sub_directory_name)
print(f"  - 모델 저장 기본 위치를 '{model_file_path}'로 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 모델 저장 함수 정의
def save_model(model, model_name, target, features):
    model_filename = f'{model_file_path}/{model_name}_{target}.pkl'
    joblib.dump(model, model_filename)
    print(f"  - 모델을 '{model_filename}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    # 사용된 피처 이름 저장
    features_filename = f'{model_file_path}/{model_name}_{target}_features.pkl'
    joblib.dump(features, features_filename)
    print(f"  - 모델에 사용된 피처 이름을 '{features_filename}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 모델 로드 함수 정의
def load_model(model_name, target):
    model_filename = f'{model_file_path}/{model_name}_{target}.pkl'
    model = joblib.load(model_filename)
    print(f"  - 모델({model_filename})을 읽어왔습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    # 사용된 피처 이름 로드
    features_filename = f'{model_file_path}/{model_name}_{target}_features.pkl'
    features = joblib.load(features_filename)
    print(f"  - 모델에 사용된 피처 이름({features_filename})을 읽어왔습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    return model, features

print(f"  - 라이브러리 로딩과 기본 설정을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ================================================

### 02-1-2. 모델 생성 및 학습, 평가 함수 정의

In [ ]:
# 02-1-2. 모델 생성 및 학습, 평가 함수 정의 (240703)
print("02-1-2. 모델 생성 및 학습, 평가 함수 정의")

# K-Fold 교차 검증을 통한 모델 학습 및 평가 함수 정의
def evaluate_model_with_kfold(X, y, param_grid, target_name, model_type, is_classifier=False):
    kf = KFold(n_splits=5, shuffle=True, random_state=42)  # K-Fold 교차 검증 설정
    best_params = None
    best_score = float('-inf') if is_classifier else float('inf')
    score_list = []
    metric_names = []

    param_combination_count = len(list(ParameterGrid(param_grid)))
    current_combination = 1

    for params in ParameterGrid(param_grid):
        print(f"    - {target_name} 모델 교차 검증 [{current_combination}/{param_combination_count}]")
        fold_scores = []

        for train_index, val_index in kf.split(X):
            X_train, X_val = X.iloc[train_index], X.iloc[val_index]
            y_train, y_val = y.iloc[train_index], y.iloc[val_index]

            model, *metrics = build_and_train_model(X_train, y_train, X_val, y_val, params, model_type=model_type, is_classifier=is_classifier)

            fold_scores.append(metrics)

        mean_scores = np.mean(fold_scores, axis=0)

        # 분류 모델
        if is_classifier:
            accuracy = mean_scores[0]
            if accuracy > best_score:
                best_score = accuracy
                best_params = params
        # 회귀 모델
        else:
            mse = mean_scores[0]
            if mse < best_score:
                best_score = mse
                best_params = params

        score_list.append(mean_scores)
        metric_names = ['accuracy', 'precision', 'recall', 'f1'] if is_classifier else ['mse', 'rmse', 'mae', 'r2']

        current_combination += 1

    print(f"  - {target_name} 모델 교차 검증을 완료하였습니다. [최적 파라미터: {best_params}]")

    # 최적 파라미터로 모델 학습
    best_model, *best_metrics = build_and_train_model(X, y, X, y, best_params, model_type=model_type, is_classifier=is_classifier)

    return best_model, best_params, *best_metrics, metric_names

# 모델 생성 및 학습 함수 정의
def build_and_train_model(X_train, y_train, X_val, y_val, params, model_type='DecisionTree', is_classifier=False):
    if model_type == 'DecisionTree':
        model = DecisionTreeClassifier(**params, random_state=42) if is_classifier else DecisionTreeRegressor(**params, random_state=42)
    elif model_type == 'RandomForest':
        model = RandomForestClassifier(**params, random_state=42) if is_classifier else RandomForestRegressor(**params, random_state=42)
    elif model_type == 'SVM':
        model = SVC(**params, random_state=42) if is_classifier else SVR(**params)
    elif model_type == 'XGBoost':
        model = xgb.XGBClassifier(**params, random_state=42) if is_classifier else xgb.XGBRegressor(**params, random_state=42)
    elif model_type == 'LightGBM':
        if is_classifier:
            model = lgb.LGBMClassifier(**params, random_state=42)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='logloss', callbacks=[lgb.early_stopping(stopping_rounds=10)])
        else:
            model = lgb.LGBMRegressor(**params, random_state=42)
            model.fit(X_train, y_train, eval_set=[(X_val, y_val)], eval_metric='rmse', callbacks=[lgb.early_stopping(stopping_rounds=10)])
        y_pred = model.predict(X_val)
        if is_classifier:
            accuracy = accuracy_score(y_val, y_pred)
            precision = precision_score(y_val, y_pred, average='weighted', zero_division=0)
            recall = recall_score(y_val, y_pred, average='weighted', zero_division=0)
            f1 = f1_score(y_val, y_pred, average='weighted', zero_division=0)
            return model, accuracy, precision, recall, f1
        else:
            mse = mean_squared_error(y_val, y_pred)
            rmse = np.sqrt(mse)
            mae = mean_absolute_error(y_val, y_pred)
            r2 = r2_score(y_val, y_pred)
            return model, mse, rmse, mae, r2
    elif model_type == 'CatBoost':
        model = CatBoostClassifier(**params, random_seed=42, silent=True) if is_classifier else CatBoostRegressor(**params, random_seed=42, silent=True)
    elif model_type == 'KNN':
        model = KNeighborsClassifier(**params) if is_classifier else KNeighborsRegressor(**params)

    # 모델 학습 및 예측
    model.fit(X_train, y_train)
    y_pred = model.predict(X_val)

    # 분류 모델 평가
    if is_classifier:
        accuracy = accuracy_score(y_val, y_pred)
        precision = precision_score(y_val, y_pred, average='weighted', zero_division=0)
        recall = recall_score(y_val, y_pred, average='weighted', zero_division=0)
        f1 = f1_score(y_val, y_pred, average='weighted', zero_division=0)
        return model, accuracy, precision, recall, f1
    # 회귀 모델 평가
    else:
        mse = mean_squared_error(y_val, y_pred)
        rmse = np.sqrt(mse)
        mae = mean_absolute_error(y_val, y_pred)
        r2 = r2_score(y_val, y_pred)
        return model, mse, rmse, mae, r2

# 모델 학습 및 평가 함수 정의
def train_and_evaluate_models(model_type, param_grid_regressor, param_grid_classifier):
    # 데이터프레임 복사
    train_df_copy = train_df.copy()
    predict_df_copy = predict_df.copy()
    print(f"  - 데이터프레임을 복사하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    X_train_stroke = train_df_copy[features_stroke]
    X_train_penalty = train_df_copy[features_penalty]
    X_train_putt = train_df_copy[features_putt]
    X_train_fw_hit = train_df_copy[train_df_copy['par'] != 3][features_fw_hit]

    y_train_stroke = train_df_copy['stroke']
    y_train_penalty = train_df_copy['penalty']
    y_train_putt = train_df_copy['putt']
    y_train_fw_hit = train_df_copy.loc[train_df_copy['par'] != 3, 'fw_hit']

    # K-Fold 교차 검증 설정
    kf = KFold(n_splits=5, shuffle=True, random_state=42)
    print(f"  - K-Fold 교차 검증을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    model_names = ['Stroke', 'Penalty', 'Putt', 'FW_Hit']
    X_trains = [X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit]
    y_trains = [y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit]
    param_grids = [param_grid_regressor, param_grid_regressor, param_grid_regressor, param_grid_classifier]
    is_classifier = [False, False, False, True]

    best_params_dict = {}
    evaluation_results = {}
    train_valid_scores = {}

    for model_name, X_train, y_train, param_grid, classifier in zip(model_names, X_trains, y_trains, param_grids, is_classifier):
        print(f"\n  - {model_name} 모델을 학습 및 평가합니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
        model, best_params, *metrics, metric_names = evaluate_model_with_kfold(X_train, y_train, param_grid, model_name.capitalize(), model_type=model_type, is_classifier=classifier)

        # 평가 결과 저장
        results[model_name][model_type] = {'최적 파라미터': best_params, **dict(zip(metric_names, metrics))}
        save_model(model, model_type, model_name, eval(f'features_{model_name.lower()}'))

        best_params_dict[model_name] = best_params
        evaluation_results[model_name] = dict(zip(metric_names, metrics))

        # 평가 결과를 파일로 저장
        results_filename = f"{model_file_path}/{model_type}_{model_name}_evaluation.json"
        with open(results_filename, 'w') as f:
            json.dump(evaluation_results[model_name], f)
        print(f"  - 평가 결과를 '{results_filename}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

        # 최적 파라미터 저장
        params_filename = f"{model_file_path}/{model_type}_{model_name}_params.json"
        with open(params_filename, 'w') as f:
            json.dump(best_params_dict[model_name], f)
        print(f"  - 최적 파라미터를 '{params_filename}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

        # 최적 파라미터로 모델 학습 후 성능 비교
        X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train, y_train, test_size=0.2, random_state=42)
        train_score, valid_score = print_train_valid_scores(model, X_train_split, y_train_split, X_val_split, y_val_split)
        train_valid_scores[model_name] = {'훈련 데이터 점수': train_score, '검증 데이터 점수': valid_score}

        # 훈련 및 검증 데이터 점수 저장
        scores_filename = f"{model_file_path}/{model_type}_{model_name}_scores.json"
        with open(scores_filename, 'w') as f:
            json.dump(train_valid_scores[model_name], f)
        print(f"  - 훈련 및 검증 데이터 점수를 '{scores_filename}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

        print(F"\n  - {model_name} 모델의 훈련 및 검증 데이터의 점수:")
        print(f"    - 훈련 데이터 점수: {train_score}")
        print(f"    - 검증 데이터 점수: {valid_score}")

    print(f"\n  - {model_type} 모델 학습 및 평가가 완료되었습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

    print("\n  - 각 변수별 최적 파라미터:")
    for model_name, best_params in best_params_dict.items():
        print(f"    - {model_name}: {best_params}")

    print("\n  - 모델 평가 결과:")
    for model_name, eval_result in evaluation_results.items():
        print(f"    - {model_name}:")
        for metric_name, metric_value in eval_result.items():
            print(f"      - {metric_name.upper()}: {metric_value}")

    print("\n  - 모델 훈련 및 검증 데이터 점수:")
    for model_name, scores in train_valid_scores.items():
        print(f"    - {model_name}:")
        for score_type, score_value in scores.items():
            print(f"      - {score_type}: {score_value}")

    return best_params_dict, evaluation_results, train_valid_scores

def print_train_valid_scores(model, X_train, y_train, X_valid, y_valid):
    if hasattr(model, 'predict_proba'):
        # 분류 모델의 경우
        train_score = accuracy_score(y_train, model.predict(X_train))
        valid_score = accuracy_score(y_valid, model.predict(X_valid))
    else:
        # 회귀 모델의 경우
        train_score = model.score(X_train, y_train)
        valid_score = model.score(X_valid, y_valid)

    return train_score, valid_score

print(f"  - 모델 생성 및 학습, 평가 함수 정의를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

### 02-1-3. 시각화 함수 정의

In [ ]:
# 02-1-3. 시각화 함수 정의 (240701)
print("02-1-3. 시각화 함수 정의")

# 모델 시각화 함수 정의
def plot_predictions(y_true, y_pred, title, xlabel, ylabel):
    plt.scatter(y_true, y_pred, alpha=0.1)
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.title(title)
    plt.plot([min(y_true), max(y_true)], [min(y_true), max(y_true)], 'r--')

# 모델 예측 결과 시각화 함수 정의
def visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                          X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                          y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit):
    y_pred_stroke = model_stroke.predict(X_train_stroke)
    y_pred_penalty = model_penalty.predict(X_train_penalty)
    y_pred_putt = model_putt.predict(X_train_putt)
    y_pred_fw_hit = model_fw_hit.predict(X_train_fw_hit)

    plt.figure(figsize=(10, 10))

    plt.subplot(2, 2, 1)
    plot_predictions(y_train_stroke, y_pred_stroke, 'Stroke Prediction', 'Actual Stroke', 'Predicted Stroke')

    plt.subplot(2, 2, 2)
    plot_predictions(y_train_penalty, y_pred_penalty, 'Penalty Prediction', 'Actual Penalty', 'Predicted Penalty')

    plt.subplot(2, 2, 3)
    plot_predictions(y_train_putt, y_pred_putt, 'Putt Prediction', 'Actual Putt', 'Predicted Putt')

    plt.subplot(2, 2, 4)
    y_pred_fw_hit_class = (y_pred_fw_hit > 0.5).astype(int)  # 이진 분류로 변환
    cm = confusion_matrix(y_train_fw_hit, y_pred_fw_hit_class)
    disp = ConfusionMatrixDisplay(confusion_matrix=cm)
    disp.plot(ax=plt.gca())
    plt.title("FW Hit Prediction - Confusion Matrix")

    plt.tight_layout()
    plt.show()

# SHAP 값 계산 및 시각화 함수 정의
def plot_shap_summary(model, X_test, title, subplot, plot_type="bar"):
    explainer = shap.Explainer(model)
    shap_values = explainer(X_test)
    plt.subplot(2, 2, subplot)
    shap.summary_plot(shap_values, X_test, plot_type=plot_type, show=False)
    plt.title(title, fontsize=15)
    plt.xticks(fontsize=10)
    plt.yticks(fontsize=10)

# SHAP 값 시각화 함수 정의
def visualize_shap_values(model_stroke, model_penalty, model_putt, model_fw_hit,
                          X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit):
    plt.figure(figsize=(20, 14))
    plot_shap_summary(model_stroke, X_train_stroke, 'Stroke SHAP Values', 1, "bar")
    plot_shap_summary(model_penalty, X_train_penalty, 'Penalty SHAP Values', 2, "bar")
    plot_shap_summary(model_putt, X_train_putt, 'Putt SHAP Values', 3, "bar")
    plot_shap_summary(model_fw_hit, X_train_fw_hit, 'FW Hit SHAP Values', 4, "bar")
    plt.tight_layout()
    plt.show()

    plt.figure(figsize=(20, 14))
    plot_shap_summary(model_stroke, X_train_stroke, 'Stroke SHAP Values', 1, "dot")
    plot_shap_summary(model_penalty, X_train_penalty, 'Penalty SHAP Values', 2, "dot")
    plot_shap_summary(model_putt, X_train_putt, 'Putt SHAP Values', 3, "dot")
    plot_shap_summary(model_fw_hit, X_train_fw_hit, 'FW Hit SHAP Values', 4, "dot")
    plt.tight_layout()
    plt.show()

# 실제 예측 수행 함수 정의
def predict_and_update(model, features, target_col, df, additional_updates=None):
    """
    주어진 모델과 피처를 사용하여 예측을 수행하고 데이터프레임을 업데이트합니다.

    Parameters:
    model: 예측 모델
    features: 예측에 사용되는 피처 목록
    target_col: 예측 결과를 저장할 대상 열
    df: 예측을 수행할 데이터프레임
    additional_updates: 추가로 업데이트할 함수 리스트 (Optional)
    """
    X_predict = df[features].dropna()
    predictions = model.predict(X_predict)
    df.loc[X_predict.index, target_col] = predictions

    if additional_updates:
        for update_func in additional_updates:
            update_func(df)

def update_score_minus_stroke(df):
    df['score_minus_stroke'] = df['score'] - df['stroke']

def update_score_minus_stroke_minus_penalty(df):
    df['score_minus_stroke_minus_penalty'] = df['score_minus_stroke'] - df['penalty']

# 학습과 예측에 사용할 변수와 예측할 변수 설정
features_stroke = ['no', 'hole_no', 'par', 'month', 'hour', 'season', 'course_type_b9', 'score']
features_penalty = ['no', 'hole_no', 'par', 'month', 'hour', 'season', 'course_type_b9', 'score', 'stroke', 'score_minus_stroke']
features_putt = ['no', 'hole_no', 'par', 'month', 'hour', 'season', 'course_type_b9', 'score', 'stroke', 'penalty', 'score_minus_stroke_minus_penalty']
features_fw_hit = ['no', 'hole_no', 'par', 'month', 'hour', 'season', 'course_type_b9', 'stroke', 'penalty', 'putt']
print(f"  - 학습과 예측에 사용할 변수와 예측할 변수를 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 평가 결과 저장할 딕셔너리 생성
results = {
    'Stroke': {},
    'Penalty': {},
    'Putt': {},
    'FW_Hit': {}
}

print(f"  - 시각화 함수 정의를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

## 02-2. 의사결정 트리 (Decision Tree)

In [ ]:
# 02-2-1. 의사결정 트리 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("02-2-1. 의사결정 트리 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'max_depth': [2, 3, 4, 5],  # 트리의 최대 깊이. 너무 깊으면 과적합 위험이 있음.
    'min_samples_leaf': [2, 3],  # 리프 노드가 되기 위한 최소 샘플 수. 너무 낮으면 과적합 위험이 있음.
    'min_samples_split': [2, 3, 4]  # 분할하기 위한 최소 샘플 수. 너무 낮으면 과적합 위험이 있음.
}
param_grid_classifier = {
    'max_depth': [2, 3, 4, 5],
    'min_samples_leaf': [2, 3],
    'min_samples_split': [2, 3, 4]
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 의사결정 트리 모델 학습 및 평가
train_and_evaluate_models('DecisionTree', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-2-2. 의사결정 트리 모델 시각화 (240628)
print("02-2-2. 의사결정 트리 모델 시각화")

from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay

# 모델 불러오기
model_stroke, _ = load_model('DecisionTree', 'Stroke')
model_penalty, _ = load_model('DecisionTree', 'Penalty')
model_putt, _ = load_model('DecisionTree', 'Putt')
model_fw_hit, _ = load_model('DecisionTree', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-2-3. 의사결정 트리 모델을 이용한 실제 예측 수행 (240628)
print("02-2-3. 의사결정 트리 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('DecisionTree', 'Stroke')
model_penalty, features_penalty = load_model('DecisionTree', 'Penalty')
model_putt, features_putt = load_model('DecisionTree', 'Putt')
model_fw_hit, features_fw_hit = load_model('DecisionTree', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

## 02-3. 랜덤 포레스트 (Random Forest)

In [ ]:
# 02-3-1. 랜덤 포레스트 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("12-1. 랜덤 포레스트 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'max_depth': [5, 7, 10],  # 트리의 최대 깊이. 너무 깊으면 과적합 위험이 있음.
    'min_samples_leaf': [2, 3, 4],  # 리프 노드가 되기 위한 최소 샘플 수. 너무 낮으면 과적합 위험이 있음.
    'min_samples_split': [2, 10, 15],  # 분할하기 위한 최소 샘플 수. 너무 낮으면 과적합 위험이 있음.
    'n_estimators': [100, 150, 200]  # 트리의 개수. 너무 많으면 과적합 위험이 있음.
}
param_grid_classifier = {
    'max_depth': [3, 7, 10],
    'min_samples_leaf': [2, 3, 4],
    'min_samples_split': [2, 10, 15],
    'n_estimators': [100, 150, 200]
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 랜덤 포레스트 모델 학습 및 평가
train_and_evaluate_models('RandomForest', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-3-2. 랜덤 포레스트 모델 시각화 (240628)
print("12-2. 랜덤 포레스트 모델 시각화")

# 모델 불러오기
model_stroke, _ = load_model('RandomForest', 'Stroke')
model_penalty, _ = load_model('RandomForest', 'Penalty')
model_putt, _ = load_model('RandomForest', 'Putt')
model_fw_hit, _ = load_model('RandomForest', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-3-3. 랜덤 포레스트 모델을 이용한 실제 예측 수행 (240628)
print("02-3-3. 랜덤 포레스트 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('RandomForest', 'Stroke')
model_penalty, features_penalty = load_model('RandomForest', 'Penalty')
model_putt, features_putt = load_model('RandomForest', 'Putt')
model_fw_hit, features_fw_hit = load_model('RandomForest', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

## 02-4. 서포트 벡터 머신 (SVM; Support Vector Machine)

In [ ]:
# 02-4-1. 서포트 벡터 머신 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("02-4-1. 서포트 벡터 머신 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'C': [0.1, 1, 2, 5],  # Regularization parameter. 너무 크면 과적합 위험이 있음.
    'epsilon': [0.2, 0.5, 1.0],  # Epsilon in the epsilon-SVR model. 너무 작으면 과적합 위험이 있음.
    'kernel': ['linear', 'poly', 'rbf']  # 데이터의 변환 방식. 고차원 커널은 과적합 위험이 있음.
}
param_grid_classifier = {
    'C': [0.1, 1, 2, 5],
    'kernel': ['linear', 'poly', 'rbf']
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 서포트 벡터 머신 모델 학습 및 평가
train_and_evaluate_models('SVM', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-4-2. 서포트 벡터 머신 모델 시각화 (240628)
print("13-2. 서포트 벡터 머신 모델 시각화")

# 모델 불러오기
model_stroke, _ = load_model('SVM', 'Stroke')
model_penalty, _ = load_model('SVM', 'Penalty')
model_putt, _ = load_model('SVM', 'Putt')
model_fw_hit, _ = load_model('SVM', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-4-3. 서포트 벡터 머신 모델을 이용한 실제 예측 수행 (240628)
print("02-4-3. 서포트 벡터 머신 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('SVM', 'Stroke')
model_penalty, features_penalty = load_model('SVM', 'Penalty')
model_putt, features_putt = load_model('SVM', 'Putt')
model_fw_hit, features_fw_hit = load_model('SVM', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

## 02-5. XGBoost (eXtreme Gradient Boosting)

In [ ]:
# 02-5-1. XGBoost 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("02-5-1. XGBoost 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'colsample_bytree': [0.1, 0.5, 1.0],  # 트리를 구성할 때 사용할 특징의 비율. 너무 높으면 과적합 위험이 있음.
    'gamma': [1e-9, 0.1, 0.5],  # 노드 분할에 필요한 최소 손실 감소. 너무 작으면 과적합 위험이 있음.
    'learning_rate': [0.01, 0.1],  # 학습률. 너무 높으면 과적합 위험이 있음.
    'max_depth': [2, 3],  # 트리의 최대 깊이. 너무 깊으면 과적합 위험이 있음.
    'min_child_weight': [0, 5, 10],  # 리프 노드가 되기 위한 최소 가중치 합. 너무 작으면 과적합 위험이 있음.
    'n_estimators': [100, 500, 1000],  # 부스팅 단위의 개수. 너무 많으면 과적합 위험이 있음.
    'subsample': [0.1, 0.5, 1.0]  # 트리를 구성할 때 사용할 데이터의 비율. 너무 높으면 과적합 위험이 있음.
}
param_grid_classifier = {
    'colsample_bytree': [0.1, 0.5, 1.0],
    'gamma': [1e-9, 0.1, 0.5],
    'learning_rate': [0.01, 0.1],
    'max_depth': [2, 3],
    'min_child_weight': [0, 5, 10],
    'n_estimators': [100, 500, 1000],
    'subsample': [0.1, 0.5, 1.0]
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# XGBoost 모델 학습 및 평가
train_and_evaluate_models('XGBoost', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-5-2. XGBoost 모델 시각화 (240628)
print("02-5-2. XGBoost 모델 시각화")

# 모델 불러오기
model_stroke, _ = load_model('XGBoost', 'Stroke')
model_penalty, _ = load_model('XGBoost', 'Penalty')
model_putt, _ = load_model('XGBoost', 'Putt')
model_fw_hit, _ = load_model('XGBoost', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

# SHAP 값 시각화
visualize_shap_values(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-5-3. XGBoost 모델을 이용한 실제 예측 수행 (240628)
print("02-5-3. XGBoost 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('XGBoost', 'Stroke')
model_penalty, features_penalty = load_model('XGBoost', 'Penalty')
model_putt, features_putt = load_model('XGBoost', 'Putt')
model_fw_hit, features_fw_hit = load_model('XGBoost', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

## 02-6. LightGBM (Light Gradient Boosting Machine)

In [ ]:
# 02-6-1. LightGBM 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("15-1. LightGBM 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'learning_rate': [0.01, 0.05, 0.1],  # 학습률. 너무 높으면 과적합 위험이 있음.
    'max_depth': [2, 3, 4],  # 트리의 최대 깊이. 너무 깊으면 과적합 위험이 있음.
    'min_child_samples': [20, 30, 50],  # 리프 노드가 되기 위한 최소 샘플 수. 너무 작으면 과적합 위험이 있음
    'n_estimators': [100, 500],  # 트리의 개수. 너무 많으면 과적합 위험이 있음.
    'num_leaves': [32]  # 하나의 트리가 가질 수 있는 최대 리프의 수.
}
param_grid_classifier = {
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [2, 3, 4],
    'min_child_samples': [20, 30, 50],
    'n_estimators': [100, 500],
    'num_leaves': [32]
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# LightGBM 모델 학습 및 평가
train_and_evaluate_models('LightGBM', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-6-2. LightGBM 모델 시각화 (240628)
print("02-6-2. LightGBM 모델 시각화")

# 모델 불러오기
model_stroke, _ = load_model('LightGBM', 'Stroke')
model_penalty, _ = load_model('LightGBM', 'Penalty')
model_putt, _ = load_model('LightGBM', 'Putt')
model_fw_hit, _ = load_model('LightGBM', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

# SHAP 값 시각화
visualize_shap_values(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-6-3. LightGBM 모델을 이용한 실제 예측 수행 (240628)
print("02-6-3. LightGBM 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('LightGBM', 'Stroke')
model_penalty, features_penalty = load_model('LightGBM', 'Penalty')
model_putt, features_putt = load_model('LightGBM', 'Putt')
model_fw_hit, features_fw_hit = load_model('LightGBM', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

## 02-7. CatBoost (Category Boosting)

In [ ]:
# 02-7-1. CatBoost 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("16-1. CatBoost 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'depth': [2, 4, 5],  # 각 트리의 깊이. 너무 깊으면 과적합 위험이 있음.
    'iterations': [100, 500, 1000],  # 부스팅 반복 횟수. 너무 많으면 과적합 위험이 있음.
    'l2_leaf_reg': [1e-6, 1e-3, 1, 10],  # L2 정규화 계수. 너무 작으면 과적합 위험이 있음.
    'learning_rate': [0.01, 0.05, 0.10]  # 학습률. 너무 높으면 과적합 위험이 있음.
}
param_grid_classifier = {
    'depth': [2, 4, 5],
    'iterations': [100, 500, 1000],
    'l2_leaf_reg': [1e-6, 1e-3, 1, 10],
    'learning_rate': [0.01, 0.05, 0.10]
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# CatBoost 모델 학습 및 평가
train_and_evaluate_models('CatBoost', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-7-2. CatBoost 모델 시각화 (240628)
print("02-7-2. CatBoost 모델 시각화")

# 모델 불러오기
model_stroke, _ = load_model('CatBoost', 'Stroke')
model_penalty, _ = load_model('CatBoost', 'Penalty')
model_putt, _ = load_model('CatBoost', 'Putt')
model_fw_hit, _ = load_model('CatBoost', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

# SHAP 값 시각화
visualize_shap_values(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-7-3. CatBoost 모델을 이용한 실제 예측 수행 (240628)
print("16-3. CatBoost 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('CatBoost', 'Stroke')
model_penalty, features_penalty = load_model('CatBoost', 'Penalty')
model_putt, features_putt = load_model('CatBoost', 'Putt')
model_fw_hit, features_fw_hit = load_model('CatBoost', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

## 02-8. K-최근접 이웃 (KNN; K-Nearest Neighbors)

In [ ]:
# 02-8-1. K-Nearest Neighbors 모델을 이용한 모델 생성, 학습 및 평가 (240628)
print("02-8-1. K-Nearest Neighbors 모델을 이용한 모델 생성, 학습 및 평가")

# 하이퍼파라미터 튜닝
param_grid_regressor = {
    'n_neighbors': [3, 4, 5, 6, 7],  # 이웃의 수. 너무 작으면 과적합 위험이 있음.
    'p': [1, 2],  # 거리 측정 방식. 1: 맨해튼 거리, 2: 유클리드 거리.
#    'weights': ['uniform', 'distance']  # 가중치 함수. 'uniform'은 동일한 가중치, 'distance'는 거리의 역수에 비례한 가중치를 사용.
    'weights': ['uniform']  # 가중치 함수. 'uniform'은 동일한 가중치, 'distance'는 거리의 역수에 비례한 가중치를 사용.
}
param_grid_classifier = {
    'n_neighbors': [3, 4, 5, 6, 7],
    'p': [1, 2],
#    'weights': ['uniform', 'distance']
    'weights': ['uniform']
}
print(f"  - 하이퍼파라미터 튜닝을 설정하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# KNN 모델 학습 및 평가
train_and_evaluate_models('KNN', param_grid_regressor, param_grid_classifier)

print(f"\n  - 모델 학습 및 평가를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-8-2. K-Nearest Neighbors 모델 시각화 (240628)
print("02-8-2. K-Nearest Neighbors 모델 시각화")

# 모델 불러오기
model_stroke, _ = load_model('KNN', 'Stroke')
model_penalty, _ = load_model('KNN', 'Penalty')
model_putt, _ = load_model('KNN', 'Putt')
model_fw_hit, _ = load_model('KNN', 'FW_Hit')

# 모델 예측 결과 시각화
visualize_predictions(model_stroke, model_penalty, model_putt, model_fw_hit,
                      X_train_stroke, X_train_penalty, X_train_putt, X_train_fw_hit,
                      y_train_stroke, y_train_penalty, y_train_putt, y_train_fw_hit)

print(f"  - 시각화를 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 02-8-3. K-Nearest Neighbors 모델을 이용한 실제 예측 수행 (240628)
print("02-8-3. K-Nearest Neighbors 모델을 이용한 실제 예측 수행")

# 모델 불러오기
model_stroke, features_stroke = load_model('KNN', 'Stroke')
model_penalty, features_penalty = load_model('KNN', 'Penalty')
model_putt, features_putt = load_model('KNN', 'Putt')
model_fw_hit, features_fw_hit = load_model('KNN', 'FW_Hit')

# predict_df_copy 변수 초기화
predict_df_copy = predict_df.copy()

# stroke 예측 및 업데이트
predict_and_update(model_stroke, features_stroke, 'stroke', predict_df_copy, [update_score_minus_stroke])

# penalty 예측 및 업데이트 (stroke 포함)
predict_and_update(model_penalty, features_penalty, 'penalty', predict_df_copy, [update_score_minus_stroke_minus_penalty])

# putt 예측 및 업데이트 (stroke와 penalty 포함)
predict_and_update(model_putt, features_putt, 'putt', predict_df_copy)

# fw_hit 예측 및 업데이트 (stroke, penalty, putt 포함)
predict_and_update(model_fw_hit, features_fw_hit, 'fw_hit', predict_df_copy)

print(f"\n  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

# 03. 모델 평가

In [ ]:
# 03-1. 모델 평가 결과 종합 및 저장 (240707)
print("03-1. 모델 평가 결과 종합 및 저장")

import pandas as pd
import json

# 파일 경로 설정
sub_directory_name = 'Output'
output_file_name = 'gScore_Model_Evaluation_Results.parquet'
output_file_path = os.path.join(base_path, sub_directory_name, output_file_name)

# 모델 평가 결과 종합 및 저장
model_types = ['DecisionTree', 'RandomForest', 'SVM', 'XGBoost', 'LightGBM', 'CatBoost', 'KNN']
model_names = ['Stroke', 'Penalty', 'Putt', 'FW_Hit']
results_dict = {}
params_dict = {}
scores_dict = {}

for model_name in model_names:
    for model_type in model_types:
        # 평가 결과 로드
        results_filename = f"{model_file_path}/{model_type}_{model_name}_evaluation.json"
        if os.path.exists(results_filename):
            with open(results_filename, 'r') as f:
                metrics = json.load(f)
                results_dict.setdefault(model_name, {}).update({model_type: metrics})

        # 최적 파라미터 로드
        params_filename = f"{model_file_path}/{model_type}_{model_name}_params.json"
        if os.path.exists(params_filename):
            with open(params_filename, 'r') as f:
                params = json.load(f)
                params_dict.setdefault(model_name, {}).update({model_type: params})

        # 훈련 및 검증 데이터 점수 로드
        scores_filename = f"{model_file_path}/{model_type}_{model_name}_scores.json"
        if os.path.exists(scores_filename):
            with open(scores_filename, 'r') as f:
                scores = json.load(f)
                scores_dict.setdefault(model_name, {}).update({model_type: scores})

# 매트릭 이름 설정
metric_columns = {
    'Stroke': ['Model', 'ModelType', 'MSE', 'RMSE', 'MAE', 'R2'],
    'Penalty': ['Model', 'ModelType', 'MSE', 'RMSE', 'MAE', 'R2'],
    'Putt': ['Model', 'ModelType', 'MSE', 'RMSE', 'MAE', 'R2'],
    'FW_Hit': ['Model', 'ModelType', 'Accuracy', 'Precision', 'Recall', 'F1']
}

# 결과를 DataFrame으로 변환
results_df = pd.concat(
    [pd.DataFrame([(model, mt, *m.values()) for mt, m in data.items()], columns=metric_columns[model]) for model, data in results_dict.items()]
).reset_index(drop=True)

params_df = pd.DataFrame([(model, mt, p) for model, data in params_dict.items() for mt, p in data.items()],
                         columns=['Model', 'ModelType', 'Params'])

scores_df = pd.DataFrame([(model, mt, *s.values()) for model, data in scores_dict.items() for mt, s in data.items()],
                         columns=['Model', 'ModelType', 'TrainScore', 'ValidScore'])

# 결과 출력 (소수점 3째자리로 강제 표시하여 출력)
def format_float(val):
    try:
        return f"{float(val):.3f}"
    except ValueError:
        return val

for model_name in model_names:
    print(f"  * {model_name}")
    print(f"    - {model_name} 평가 결과:")
    if model_name in ['Stroke', 'Penalty', 'Putt']:
        df = results_df[results_df['Model'] == model_name][['ModelType', 'MSE', 'RMSE', 'MAE', 'R2']].applymap(format_float)
        display(df)
    elif model_name == 'FW_Hit':
        df = results_df[results_df['Model'] == model_name][['ModelType', 'Accuracy', 'Precision', 'Recall', 'F1']].applymap(format_float)
        display(df)

    print(f"    - {model_name} 최적 파라미터:")
    display(params_df[params_df['Model'] == model_name])

    print(f"    - {model_name} 훈련 및 검증 데이터 점수:")
    df = scores_df[scores_df['Model'] == model_name].applymap(format_float)
    display(df)

# 평가 결과 저장
results_df.to_parquet(output_file_path)

# 최적 파라미터와 점수 저장
params_output_file_name = 'gScore_Model_Params.parquet'
params_output_file_path = os.path.join(base_path, sub_directory_name, params_output_file_name)
params_df.to_parquet(params_output_file_path)

scores_output_file_name = 'gScore_Model_Scores.parquet'
scores_output_file_path = os.path.join(base_path, sub_directory_name, scores_output_file_name)
scores_df.to_parquet(scores_output_file_path)

print(f"  - 모델 평가 결과를 '{output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
print(f"  - 최적 파라미터를 '{params_output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")
print(f"  - 훈련 및 검증 데이터 점수를 '{scores_output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 03-2. 모델별 최적 파라미터 테이블 생성 및 저장 (240703)
print("03-2. 모델별 최적 파라미터 테이블 생성 및 저장")

import pandas as pd
import json

# 파일 경로 설정
sub_directory_name = 'Output'
output_file_name = 'gScore_Model_Params_Tables.parquet'
output_file_path = os.path.join(base_path, sub_directory_name, output_file_name)

# 모델별 최적 파라미터 테이블 생성
model_types = ['DecisionTree', 'RandomForest', 'SVM', 'XGBoost', 'LightGBM', 'CatBoost', 'KNN']
model_names = ['Stroke', 'Penalty', 'Putt', 'FW_Hit']
param_tables = {}

for model_type in model_types:
    param_dict = {}
    for model_name in model_names:
        # 최적 파라미터 로드
        params_filename = f"{model_file_path}/{model_type}_{model_name}_params.json"
        if os.path.exists(params_filename):
            with open(params_filename, 'r') as f:
                params = json.load(f)
                param_dict[model_name] = params

    # 최적 파라미터를 DataFrame으로 변환
    if param_dict:
        param_df = pd.DataFrame(param_dict).T
        param_tables[model_type] = param_df

# 결과 출력
for model_type, param_df in param_tables.items():
    print(f"  * {model_type} 최적 파라미터:")
    display(param_df)

# 최적 파라미터 테이블 저장
param_tables_df = pd.concat(param_tables, keys=param_tables.keys()).reset_index(level=0).rename(columns={'level_0': 'ModelType'})
param_tables_df.to_parquet(output_file_path)

print(f"  - 모델별 최적 파라미터 테이블을 '{output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

In [ ]:
# 03-3. 모델 평가 결과 시각화 (240703)
print("03-3. 모델 평가 결과 시각화")

import seaborn as sns
import matplotlib.pyplot as plt

# 시각화 함수 정의
def plot_metrics(metric, title, ylabel, data, ax, is_max_best=True, show_legend=True):
    sns.barplot(data=data, x='ModelType', y=metric, hue='Model', ax=ax)
    ax.set_title(title)
    ax.set_ylabel(ylabel)
    ax.set_xlabel('ModelType')
    if show_legend:
        ax.legend(title='Model', loc='upper right')
    else:
        ax.legend_.remove()
    # 최고 또는 최저값에 가로선 추가
    if is_max_best:
        best_value = data[metric].max()
    else:
        best_value = data[metric].min()
    ax.axhline(best_value, color='red', linestyle='--')

# Stroke에 대한 그래프
print("  * Stroke")
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

stroke_data = results_df[results_df['Model'] == 'Stroke']
plot_metrics('MSE', 'Stroke - Mean Squared Error (MSE)', 'MSE', stroke_data, axs[0, 0], is_max_best=False, show_legend=False)
plot_metrics('RMSE', 'Stroke - Root Mean Squared Error (RMSE)', 'RMSE', stroke_data, axs[0, 1], is_max_best=False, show_legend=False)
plot_metrics('MAE', 'Stroke - Mean Absolute Error (MAE)', 'MAE', stroke_data, axs[1, 0], is_max_best=False, show_legend=False)
plot_metrics('R2', 'Stroke - R-Squared (R2)', 'R2', stroke_data, axs[1, 1], is_max_best=True, show_legend=False)

plt.tight_layout()
plt.show()

# Penalty에 대한 그래프
print(f"\n  * Penalty")
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

penalty_data = results_df[results_df['Model'] == 'Penalty']
plot_metrics('MSE', 'Penalty - Mean Squared Error (MSE)', 'MSE', penalty_data, axs[0, 0], is_max_best=False, show_legend=False)
plot_metrics('RMSE', 'Penalty - Root Mean Squared Error (RMSE)', 'RMSE', penalty_data, axs[0, 1], is_max_best=False, show_legend=False)
plot_metrics('MAE', 'Penalty - Mean Absolute Error (MAE)', 'MAE', penalty_data, axs[1, 0], is_max_best=False, show_legend=False)
plot_metrics('R2', 'Penalty - R-Squared (R2)', 'R2', penalty_data, axs[1, 1], is_max_best=True, show_legend=False)

plt.tight_layout()
plt.show()

# Putt에 대한 그래프
print(f"\n  * Putt")
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

putt_data = results_df[results_df['Model'] == 'Putt']
plot_metrics('MSE', 'Putt - Mean Squared Error (MSE)', 'MSE', putt_data, axs[0, 0], is_max_best=False, show_legend=False)
plot_metrics('RMSE', 'Putt - Root Mean Squared Error (RMSE)', 'RMSE', putt_data, axs[0, 1], is_max_best=False, show_legend=False)
plot_metrics('MAE', 'Putt - Mean Absolute Error (MAE)', 'MAE', putt_data, axs[1, 0], is_max_best=False, show_legend=False)
plot_metrics('R2', 'Putt - R-Squared (R2)', 'R2', putt_data, axs[1, 1], is_max_best=True, show_legend=False)

plt.tight_layout()
plt.show()

# FW Hit에 대한 그래프
print(f"\n  * FW Hit")
fig, axs = plt.subplots(2, 2, figsize=(14, 12))

fw_hit_data = results_df[results_df['Model'] == 'FW_Hit']
plot_metrics('Accuracy', 'FW Hit - Accuracy', 'Accuracy', fw_hit_data, axs[0, 0], is_max_best=True, show_legend=False)
plot_metrics('Precision', 'FW Hit - Precision', 'Precision', fw_hit_data, axs[0, 1], is_max_best=True, show_legend=False)
plot_metrics('Recall', 'FW Hit - Recall', 'Recall', fw_hit_data, axs[1, 0], is_max_best=True, show_legend=False)
plot_metrics('F1', 'FW Hit - F1 Score', 'F1 Score', fw_hit_data, axs[1, 1], is_max_best=True, show_legend=False)

plt.tight_layout()
plt.show()

# ==================================================

In [ ]:
# 03-4. 종합 평가 결과 및 모델 추천 (240707)
print("03-4. 종합 평가 결과 및 모델 추천")

import json

# 평가 결과 불러오기
sub_directory_name = 'Output'
results_file_path = os.path.join(base_path, sub_directory_name, 'gScore_Model_Evaluation_Results.parquet')
scores_file_path = os.path.join(base_path, sub_directory_name, 'gScore_Model_Scores.parquet')

# 평가 결과 로드
results_df = pd.read_parquet(results_file_path)
scores_df = pd.read_parquet(scores_file_path)

# 가중치 설정
weights_classification = {'Accuracy': 0.4, 'Precision': 0.2, 'Recall': 0.2, 'F1': 0.2}
weights_regression = {'R2': 0.4, 'MSE': 0.3, 'MAE': 0.15, 'RMSE': 0.15}

# 역수 변환 함수
def inverse_weight(value, weight):
    return (1 - value) * weight

# 종합 점수 계산 함수
def calculate_composite_score(model_results, weights, inverse_metrics):
    scores = {}
    for model_name, metrics in model_results.items():
        composite_score = sum(
            inverse_weight(metrics.get(metric, 0), weight) if metric in inverse_metrics else metrics.get(metric, 0) * weight
            for metric, weight in weights.items()
        )
        scores[model_name] = composite_score
    return scores

# 모델 추천 함수
def recommend_models(results_df, model_names, weights_classification, weights_regression):
    recommendations = {}
    inverse_metrics_regression = ['MSE', 'MAE', 'RMSE']
    inverse_metrics_classification = []

    for model_name in model_names:
        if model_name == 'FW_Hit':
            scores = calculate_composite_score(
                results_df[results_df['Model'] == model_name].set_index('ModelType').to_dict('index'),
                weights_classification, inverse_metrics_classification
            )
        else:
            scores = calculate_composite_score(
                results_df[results_df['Model'] == model_name].set_index('ModelType').to_dict('index'),
                weights_regression, inverse_metrics_regression
            )

        sorted_scores = sorted(scores.items(), key=lambda x: x[1], reverse=True)
        recommendations[model_name] = [(model, score) for model, score in sorted_scores]

    return recommendations

# 과적합/과소적합 경고 추가
def add_overfitting_warning(results_df, scores_df, model_names):
    warnings = {model_name: [] for model_name in model_names}
    for model_name in model_names:
        for model_type in results_df[results_df['Model'] == model_name]['ModelType'].unique():
            train_score = round(scores_df[(scores_df['Model'] == model_name) & (scores_df['ModelType'] == model_type)]['TrainScore'].values[0], 3)
            valid_score = round(scores_df[(scores_df['Model'] == model_name) & (scores_df['ModelType'] == model_type)]['ValidScore'].values[0], 3)
            if train_score == 1.0 and valid_score == 1.0:
                warnings[model_name].append(f"{model_type} 모델: 과적합 의심 (훈련 및 검증 데이터 점수 모두 1.0)")
            elif train_score < 0.7 and valid_score < 0.7:
                warnings[model_name].append(f"{model_type} 모델: 과소적합 의심 (훈련 및 검증 데이터 점수 모두 0.7 미만)")
            elif train_score - valid_score > 0.1:
                warnings[model_name].append(f"{model_type} 모델: 과적합 의심 (검증 데이터 점수가 {(train_score - valid_score)*100:.1f}%p 낮음)")
            elif valid_score - train_score > 0.1:
                warnings[model_name].append(f"{model_type} 모델: 과소적합 의심 (검증 데이터 점수가 {(valid_score - train_score)*100:.1f}%p 높음)")
    return warnings

# 모델 추천
model_names = ['Stroke', 'Penalty', 'Putt', 'FW_Hit']
recommendations = recommend_models(results_df, model_names, weights_classification, weights_regression)

# 과적합/과소적합 경고
warnings = add_overfitting_warning(results_df, scores_df, model_names)

# 결과 출력 함수
def display_recommendations_table(recommendations, results_df, scores_df, warnings):
    recommendations_data = []
    for target, models in recommendations.items():
        for i, (model, score) in enumerate(models, 1):
            model_metrics = results_df[(results_df['Model'] == target) & (results_df['ModelType'] == model)].iloc[0].to_dict()
            train_score = round(scores_df[(scores_df['Model'] == target) & (scores_df['ModelType'] == model)]['TrainScore'].values[0], 3)
            valid_score = round(scores_df[(scores_df['Model'] == target) & (scores_df['ModelType'] == model)]['ValidScore'].values[0], 3)
            score_diff = round(abs(train_score - valid_score), 3)
            metrics_info = {key: round(value, 3) for key, value in model_metrics.items() if key not in ['Model', 'ModelType'] and pd.notna(value)}
            recommendations_data.append([target, model, round(score, 3)] + list(metrics_info.values()) + [train_score, valid_score, score_diff])

    metric_columns = {
        'Stroke': ['Target', 'Model', 'Score', 'MSE', 'RMSE', 'MAE', 'R2', 'TrainScore', 'ValidScore', 'ScoreDiff'],
        'Penalty': ['Target', 'Model', 'Score', 'MSE', 'RMSE', 'MAE', 'R2', 'TrainScore', 'ValidScore', 'ScoreDiff'],
        'Putt': ['Target', 'Model', 'Score', 'MSE', 'RMSE', 'MAE', 'R2', 'TrainScore', 'ValidScore', 'ScoreDiff'],
        'FW_Hit': ['Target', 'Model', 'Score', 'Accuracy', 'Precision', 'Recall', 'F1', 'TrainScore', 'ValidScore', 'ScoreDiff']
    }

    recommendations_df_list = []
    for target in model_names:
        target_df = pd.DataFrame([data for data in recommendations_data if data[0] == target], columns=metric_columns[target])
        recommendations_df_list.append(target_df)

    return recommendations_df_list

recommendations_df_list = display_recommendations_table(recommendations, results_df, scores_df, warnings)

# 테이블 및 경고 출력
for target_df in recommendations_df_list:
    target_name = target_df['Target'].iloc[0]
    print(f"\n  * {target_name}")
    for warning in warnings[target_name]:
        print(f"    - {warning}")
    display(target_df)

# 결과 저장
recommendations_output_file_path = os.path.join(base_path, sub_directory_name, 'Model_Recommendations.json')
with open(recommendations_output_file_path, 'w') as f:
    json.dump(recommendations, f, indent=4)
print(f"  - 모델 추천 결과를 '{recommendations_output_file_path}'에 저장하였습니다.")

# ==================================================

## 03-5. 모델 평가 결과

### 개요
다양한 머신러닝 모델을 사용하여 `Stroke`, `Penalty`, `Putt`, `FW_Hit` 네 가지 타겟 변수를 예측하는 성능을 평가하였으며, 평가에는 `MSE`, `RMSE`, `MAE`, `R2`, `Accuracy`, `Precision`, `Recall`, `F1 Score`와 같은 다양한 지표를 사용하였습니다. 각 타겟 변수에 대해 **최적의 성능을 보인 3가지 모델**을 선정하여 제시합니다.

### 평가 지표 설명

#### 회귀 모델 (Regression Models)

- **MSE (Mean Squared Error, 평균제곱오차)**: 예측 값과 실제 값의 차이를 제곱하여 평균한 값. 값이 *작을수록* 예측 정확도가 높음을 의미.
- **RMSE (Root Mean Squared Error, 평균제곱근오차)**: MSE의 제곱근으로, 예측 값과 실제 값의 차이를 직관적으로 이해하기 쉽게 변환한 값. 값이 *작을수록* 예측 정확도가 높음을 의미.
- **MAE (Mean Absolute Error, 평균절대오차)**: 예측 값과 실제 값의 차이를 절대값으로 변환하여 평균한 값. 값이 *작을수록* 예측 정확도가 높음을 의미.
- **R2 (R-Squared, 결정계수)**: 모델이 실제 값을 얼마나 잘 설명하는지를 나타내는 지표. *1에 가까울수록* 모델의 설명력이 높음을 의미.

#### 분류 모델 (Classification Models)

- **Accuracy (정확도)**: 전체 예측에서 맞춘 비율을 나타냄. 값이 *클수록* 정확도가 높음을 의미.
- **Precision (정밀도)**: 모델이 양성으로 예측한 것 중 실제 양성의 비율. 값이 *클수록* 정밀도가 높음을 의미.
- **Recall (재현율)**: 실제 양성 중 모델이 양성으로 맞춘 비율. 값이 *클수록* 재현율이 높음을 의미.
- **F1 Score**: Precision과 Recall의 조화 평균으로, 두 지표의 *균형*을 나타냄. 값이 *클수록* 성능이 균형 잡혔음을 의미.

---

### 1. Stroke

#### 모델 평가 결과
| Model          | MSE   | RMSE  | MAE   | R2    |
|----------------|-------|-------|-------|-------|
| DecisionTree   | 0.492 | 0.701 | 0.545 | 0.606 |
| RandomForest   | 0.429 | 0.655 | 0.503 | 0.656 |
| SVM            | 0.516 | 0.718 | 0.566 | 0.587 |
| XGBoost        | 0.467 | 0.683 | 0.528 | 0.626 |
| LightGBM       | 0.470 | 0.686 | 0.529 | 0.624 |
| CatBoost       | 0.364 | 0.603 | 0.472 | 0.709 |
| KNN            | 0.000 | 0.000 | 0.000 | 1.000 |

#### 훈련 및 검증 데이터 점수
| Model          | TrainScore | ValidScore | Difference |
|----------------|------------|------------|------------|
| DecisionTree   | 0.600      | 0.623      | 0.023      |
| RandomForest   | 0.651      | 0.673      | 0.022      |
| SVM            | 0.582      | 0.599      | 0.017      |
| XGBoost        | 0.620      | 0.645      | 0.025      |
| LightGBM       | 0.618      | 0.641      | 0.023      |
| CatBoost       | 0.703      | 0.725      | 0.022      |
| KNN            | 1.000      | 1.000      | 0.000      |

#### 분석 및 해석
- **CatBoost**가 MSE, RMSE, MAE에서 가장 낮은 값을 보였으며, R2 점수도 0.709로 높은 설명력을 보였습니다.
- **RandomForest** 역시 우수한 성능을 보였으며, 특히 R2가 0.656로 높았습니다.
- **KNN**은 훈련 및 검증 데이터에서 모두 완벽한 점수를 보이며 과적합이 의심됩니다.

#### 추천 모델
1. **CatBoost**: 높은 성능과 설명력을 보임.
2. **RandomForest**: 우수한 예측 성능과 안정성.
3. **XGBoost**: 전반적인 성능이 뛰어남.

---

### 2. Penalty

#### 모델 평가 결과
| Model          | MSE   | RMSE  | MAE   | R2    |
|----------------|-------|-------|-------|-------|
| DecisionTree   | 0.227 | 0.476 | 0.349 | 0.648 |
| RandomForest   | 0.208 | 0.456 | 0.333 | 0.678 |
| SVM            | 0.293 | 0.541 | 0.423 | 0.545 |
| XGBoost        | 0.218 | 0.467 | 0.341 | 0.662 |
| LightGBM       | 0.219 | 0.467 | 0.339 | 0.661 |
| CatBoost       | 0.226 | 0.475 | 0.349 | 0.649 |
| KNN            | 0.000 | 0.000 | 0.000 | 1.000 |

#### 훈련 및 검증 데이터 점수
| Model          | TrainScore | ValidScore | Difference |
|----------------|------------|------------|------------|
| DecisionTree   | 0.622      | 0.737      | 0.115      |
| RandomForest   | 0.652      | 0.765      | 0.113      |
| SVM            | 0.518      | 0.636      | 0.118      |
| XGBoost        | 0.636      | 0.752      | 0.116      |
| LightGBM       | 0.634      | 0.754      | 0.120      |
| CatBoost       | 0.624      | 0.735      | 0.111      |
| KNN            | 1.000      | 1.000      | 0.000      |

#### 분석 및 해석
- **RandomForest**가 R2가 0.678로 가장 높았으며, MSE, RMSE, MAE 모두 낮아 최상의 성능을 보였습니다.
- **CatBoost**와 **XGBoost**도 유사한 성능을 보였습니다.
- **KNN**은 훈련 및 검증 데이터에서 모두 완벽한 점수를 보이며 과적합이 의심됩니다.

#### 추천 모델
1. **RandomForest**: 전반적인 성능이 뛰어남.
2. **CatBoost**: 높은 성능과 설명력을 보임.
3. **XGBoost**: 우수한 예측 성능과 안정성.

---

### 3. Putt

#### 모델 평가 결과
| Model          | MSE    | RMSE   | MAE    | R2     |
|----------------|--------|--------|--------|--------|
| DecisionTree   | 0.000  | 0.000  | 0.000  | 1.000  |
| RandomForest   | 0.000  | 0.000  | 0.000  | 1.000  |
| SVM            | 0.005  | 0.072  | 0.046  | 0.990  |
| XGBoost        | 0.000  | 0.000  | 0.000  | 1.000  |
| LightGBM       | 0.001  | 0.033  | 0.014  | 0.998  |
| CatBoost       | 0.000  | 0.000  | 0.000  | 1.000  |
| KNN            | 0.000  | 0.000  | 0.000  | 1.000  |

#### 훈련 및 검증 데이터 점수
| Model          | TrainScore | ValidScore | Difference |
|----------------|------------|------------|------------|
| DecisionTree   | 1.000      | 1.000      | 0.000      |
| RandomForest   | 1.000      | 1.000      | 0.000      |
| SVM            | 0.990      | 0.990      | 0.000      |
| XGBoost        | 1.000      | 1.000      | 0.000      |
| LightGBM       | 0.998      | 0.998      | 0.000      |
| CatBoost       | 1.000      | 1.000      | 0.000      |
| KNN            | 1.000      | 1.000      | 0.000      |

#### 분석 및 해석
- **DecisionTree**, **RandomForest**, **XGBoost**, **CatBoost**, **KNN**이 모든 지표에서 완벽한 성능을 보였습니다.
- **SVM**은 약간의 오차가 있으나 여전히 우수한 성능을 보였습니다.

#### 추천 모델
1. **DecisionTree**: 모든 지표에서 완벽한 성능을 보임.
2. **RandomForest**: 우수한 예측 성능과 안정성.
3. **XGBoost**: 전반적인 성능이 뛰어남.

---

### 4. FW_Hit

#### 모델 평가 결과
| Model          | Accuracy | Precision | Recall | F1    |
|----------------|----------|-----------|--------|-------|
| DecisionTree   | 0.694    | 0.721     | 0.694  | 0.700 |
| RandomForest   | 0.890    | 0.890     | 0.890  | 0.889 |
| SVM            | 0.689    | 0.683     | 0.689  | 0.685 |
| XGBoost        | 0.788    | 0.784     | 0.788  | 0.785 |
| LightGBM       | 0.822    | 0.820     | 0.822  | 0.821 |
| CatBoost       | 1.000    | 1.000     | 1.000  | 1.000 |
| KNN            | 1.000    | 1.000     | 1.000  | 1.000 |

#### 훈련 및 검증 데이터 점수
| Model          | TrainScore | ValidScore | Difference |
|----------------|------------|------------|------------|
| DecisionTree   | 0.704      | 0.656      | 0.048      |
| RandomForest   | 0.893      | 0.879      | 0.014      |
| SVM            | 0.714      | 0.592      | 0.122      |
| XGBoost        | 0.803      | 0.726      | 0.077      |
| LightGBM       | 0.829      | 0.796      | 0.033      |
| CatBoost       | 1.000      | 1.000      | 0.000      |

#### 분석 및 해석
- **CatBoost**가 모든 지표에서 완벽한 성능을 보였습니다.
- **RandomForest**와 **XGBoost**도 높은 성능을 보였습니다.
- **KNN**은 훈련 데이터에서 높은 성능을 보였으나 검증 데이터에서는 다소 낮은 성능을 보이며, 과적합이 의심됩니다.

#### 추천 모델
1. **CatBoost**: 높은 성능과 설명력을 보임.
2. **RandomForest**: 우수한 예측 성능과 안정성.
3. **XGBoost**: 전반적인 성능이 뛰어남.

---

### 5. 모델별 최적 파라미터

#### DecisionTree (의사결정트리)
| Model        | max_depth | min_samples_leaf | min_samples_split |
|--------------|-----------|------------------|-------------------|
| Stroke       | 4         | 2                | 2                 |
| Penalty      | 4         | 3                | 2                 |
| Putt         | 3         | 2                | 2                 |
| FW_Hit       | 2         | 2                | 2                 |

#### RandomForest (랜덤 포레스트)
| Model        | max_depth | min_samples_leaf | min_samples_split | n_estimators |
|--------------|-----------|------------------|-------------------|--------------|
| Stroke       | 5         | 3                | 2                 | 150          |
| Penalty      | 5         | 2                | 15                | 200          |
| Putt         | 5         | 2                | 15                | 200          |
| FW_Hit       | 10        | 3                | 2                 | 200          |

#### SVM (서포트 벡터 머신)
| Model        | C   | epsilon | kernel  |
|--------------|-----|---------|---------|
| Stroke       | 1   | 0.2     | linear  |
| Penalty      | 2   | 0.2     | linear  |
| Putt         | 0.1 | 0.2     | linear  |
| FW_Hit       | 1   | -       | linear  |

#### XGBoost
| Model        | max_depth | colsample_bytree | gamma | learning_rate | min_child_weight | n_estimators | subsample |
|--------------|-----------|------------------|-------|---------------|------------------|--------------|-----------|
| Stroke       | 2         | 1.0              | 1e-09 | 0.01          | 5                | 1000         | 1.0       |
| Penalty      | 2         | 1.0              | 0.1   | 0.01          | 10               | 1000         | 1.0       |
| Putt         | 3         | 1.0              | 1e-09 | 0.1           | 0                | 1000         | 1.0       |
| FW_Hit       | 3         | 1.0              | 0.5   | 0.1           | 5                | 100          | 0.5       |

#### LightGBM
| Model        | max_depth | learning_rate | min_child_samples | n_estimators | num_leaves |
|--------------|-----------|---------------|-------------------|--------------|------------|
| Stroke       | 2         | 0.1           | 20                | 100          | 32         |
| Penalty      | 2         | 0.1           | 20                | 100          | 32         |
| Putt         | 4         | 0.1           | 20                | 500          | 32         |
| FW_Hit       | 4         | 0.1           | 30                | 100          | 32         |

#### CatBoost
| Model        | depth | iterations | l2_leaf_reg | learning_rate |
|--------------|-------|------------|-------------|---------------|
| Stroke       | 5     | 1000       | 1           | 0.01          |
| Penalty      | 2     | 1000       | 1           | 0.01          |
| Putt         | 2     | 500        | 1e-06       | 0.1           |
| FW_Hit       | 5     | 500        | 0.001       | 0.1           |

### 5. 모델별 최적 파라미터

#### KNN (K-최근접 이웃)
| Model        | n_neighbors | p   | weights   |
|--------------|-------------|-----|-----------|
| Stroke       | 5           | 1   | distance  |
| Penalty      | 7           | 2   | distance  |
| Putt         | 4           | 1   | distance  |
| FW_Hit       | 6           | 1   | distance  |

---


# 04. 최종 예측 (최종 파일: 5_final_predictions.csv)

In [ ]:
# 04-1. stroke 예측 및 정수 저장 (240630)
print("04-1. stroke 예측 및 정수 저장")

# 데이터 복사
predict_df_copy = predict_df.copy()

# 모델 불러오기
#model_stroke, features_stroke = load_model('CatBoost', 'Stroke')
model_stroke, features_stroke = load_model('RandomForest', 'Stroke')
model_penalty, features_penalty = load_model('LightGBM', 'Penalty')
#model_penalty, features_penalty = load_model('RandomForest', 'Penalty')
#model_penalty, features_penalty = load_model('XGBoost', 'Penalty')
model_putt, features_putt = load_model('DecisionTree', 'Putt')
model_fw_hit, features_fw_hit = load_model('RandomForest', 'FW_Hit')
#model_fw_hit, features_fw_hit = load_model('CatBoost', 'FW_Hit')

# 파일 경로 설정
sub_directory_name = 'Output'
output_file_name = '1_stroke_predictions.csv'
output_file_path = os.path.join(base_path, sub_directory_name, output_file_name)

# stroke 예측을 위한 데이터 준비
X_predict_stroke = predict_df_copy[features_stroke].dropna()
predicted_stroke = model_stroke.predict(X_predict_stroke)
predict_df_copy.loc[X_predict_stroke.index, 'stroke'] = np.round(predicted_stroke).astype(int)  # 예측값 정수화

print(f"  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터 저장
predict_df_copy.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"  - 데이터프레임을 '{output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

04. stroke 예측 및 정수 저장
  - 모델(/content/drive/MyDrive/Colab Notebooks/gScore/Model/RandomForest_Stroke.pkl)을 읽어왔습니다. (2024-08-08 16:34:42)
  - 모델에 사용된 피처 이름(/content/drive/MyDrive/Colab Notebooks/gScore/Model/RandomForest_Stroke_features.pkl)을 읽어왔습니다. (2024-08-08 16:34:42)
  - 모델(/content/drive/MyDrive/Colab Notebooks/gScore/Model/LightGBM_Penalty.pkl)을 읽어왔습니다. (2024-08-08 16:34:42)
  - 모델에 사용된 피처 이름(/content/drive/MyDrive/Colab Notebooks/gScore/Model/LightGBM_Penalty_features.pkl)을 읽어왔습니다. (2024-08-08 16:34:42)
  - 모델(/content/drive/MyDrive/Colab Notebooks/gScore/Model/DecisionTree_Putt.pkl)을 읽어왔습니다. (2024-08-08 16:34:42)
  - 모델에 사용된 피처 이름(/content/drive/MyDrive/Colab Notebooks/gScore/Model/DecisionTree_Putt_features.pkl)을 읽어왔습니다. (2024-08-08 16:34:42)
  - 모델(/content/drive/MyDrive/Colab Notebooks/gScore/Model/RandomForest_FW_Hit.pkl)을 읽어왔습니다. (2024-08-08 16:34:43)
  - 모델에 사용된 피처 이름(/content/drive/MyDrive/Colab Notebooks/gScore/Model/RandomForest_FW_Hit_features.pkl)을 읽어왔습니다. (2024-08-0

,date_time,no,golf_course,course,hole_no,score,...,month,hour,season,course_type_b9,score_minus_stroke,score_minus_stroke_minus_penalty
0,2021-09-10 12:06:00,1,안성베네스트,West,1,4,...,9,12,4,False,NaN,NaN
1,2021-09-10 12:06:00,1,안성베네스트,West,2,5,...,9,12,4,False,NaN,NaN
2,2021-09-10 12:06:00,1,안성베네스트,West,3,7,...,9,12,4,False,NaN,NaN
3,2021-09-10 12:06:00,1,안성베네스트,West,4,8,...,9,12,4,False,NaN,NaN
4,2021-09-10 12:06:00,1,안성베네스트,West,5,5,...,9,12,4,False,NaN,NaN
5,2021-09-10 12:06:00,1,안성베네스트,West,6,7,...,9,12,4,False,NaN,NaN
6,2021-09-10 12:06:00,1,안성베네스트,West,7,7,...,9,12,4,False,NaN,NaN
7,2021-09-10 12:06:00,1,안성베네스트,West,8,7,...,9,12,4,False,NaN,NaN
8,2021-09-10 12:06:00,1,안성베네스트,West,9,7,...,9,12,4,False,NaN,NaN
9,2021-09-10 12:06:00,1,안성베네스트,North,1,5,...,9,12,4,True,NaN,NaN


  * Prediction DataFrame Information (predict_df_copy):
<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_time                         612 non-null    datetime64[ns]
 1   no                                612 non-null    int64         
 2   golf_course                       612 non-null    object        
 3   course                            612 non-null    object        
 4   hole_no                           612 non-null    int64         
 5   score                             612 non-null    int64         
 6   stroke                            612 non-null    float64       
 7   putt                              0 non-null      float64       
 8   penalty                           0 non-null      float64       
 9   par                               612 non-null    int64         
 10 

In [ ]:
# 04-2. penalty 예측 및 정수 저장 (240630)
print("04-2. penalty 예측 및 정수 저장")

# 파일 경로 설정
output_file_name = '2_penalty_predictions.csv'
output_file_path = os.path.join(base_path, sub_directory_name, output_file_name)

# score_minus_stroke 변수 업데이트
predict_df_copy['score_minus_stroke'] = predict_df_copy['score'] - predict_df_copy['stroke']

# penalty 예측을 위한 데이터 준비
X_predict_penalty = predict_df_copy[features_penalty].dropna()
predicted_penalty = model_penalty.predict(X_predict_penalty)
predict_df_copy.loc[X_predict_penalty.index, 'penalty'] = np.round(predicted_penalty).astype(int)  # 예측값 정수화

print(f"  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터 저장
predict_df_copy.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"  - 데이터프레임을 '{output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

04-2. penalty 예측 및 정수 저장
  - 예측을 완료하였습니다. (2024-08-08 16:35:19)
  - 데이터프레임을 '/content/drive/MyDrive/Colab Notebooks/gScore/Output/2_penalty_predictions.csv'에 저장하였습니다. (2024-08-08 16:35:20)

  * Prediction DataFrame (predict_df_copy):


,date_time,no,golf_course,course,hole_no,score,...,month,hour,season,course_type_b9,score_minus_stroke,score_minus_stroke_minus_penalty
0,2021-09-10 12:06:00,1,안성베네스트,West,1,4,...,9,12,4,False,2.0,NaN
1,2021-09-10 12:06:00,1,안성베네스트,West,2,5,...,9,12,4,False,3.0,NaN
2,2021-09-10 12:06:00,1,안성베네스트,West,3,7,...,9,12,4,False,4.0,NaN
3,2021-09-10 12:06:00,1,안성베네스트,West,4,8,...,9,12,4,False,4.0,NaN
4,2021-09-10 12:06:00,1,안성베네스트,West,5,5,...,9,12,4,False,3.0,NaN
5,2021-09-10 12:06:00,1,안성베네스트,West,6,7,...,9,12,4,False,3.0,NaN
6,2021-09-10 12:06:00,1,안성베네스트,West,7,7,...,9,12,4,False,4.0,NaN
7,2021-09-10 12:06:00,1,안성베네스트,West,8,7,...,9,12,4,False,4.0,NaN
8,2021-09-10 12:06:00,1,안성베네스트,West,9,7,...,9,12,4,False,3.0,NaN
9,2021-09-10 12:06:00,1,안성베네스트,North,1,5,...,9,12,4,True,2.0,NaN


  * Prediction DataFrame Information (predict_df_copy):
<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_time                         612 non-null    datetime64[ns]
 1   no                                612 non-null    int64         
 2   golf_course                       612 non-null    object        
 3   course                            612 non-null    object        
 4   hole_no                           612 non-null    int64         
 5   score                             612 non-null    int64         
 6   stroke                            612 non-null    float64       
 7   putt                              0 non-null      float64       
 8   penalty                           612 non-null    float64       
 9   par                               612 non-null    int64         
 10 

In [ ]:
# 04-3. putt 예측 및 정수 저장 (240630)
print("04-3. putt 예측 및 정수 저장")

# 파일 경로 설정
output_file_name = '3_putt_predictions.csv'
output_file_path = os.path.join(base_path, sub_directory_name, output_file_name)

# score_minus_stroke_minus_penalty 변수 업데이트
predict_df_copy['score_minus_stroke_minus_penalty'] = predict_df_copy['score_minus_stroke'] - predict_df_copy['penalty']

# putt 예측을 위한 데이터 준비
X_predict_putt = predict_df_copy[features_putt].dropna()
predicted_putt = model_putt.predict(X_predict_putt)
predict_df_copy.loc[X_predict_putt.index, 'putt'] = np.round(predicted_putt).astype(int)  # 예측값 정수화

print(f"  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터 저장
predict_df_copy.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"  - 데이터프레임을 '{output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

04-3. putt 예측 및 정수 저장
  - 예측을 완료하였습니다. (2024-08-08 16:36:44)
  - 데이터프레임을 '/content/drive/MyDrive/Colab Notebooks/gScore/Output/3_putt_predictions.csv'에 저장하였습니다. (2024-08-08 16:36:44)

  * Prediction DataFrame (predict_df_copy):


,date_time,no,golf_course,course,hole_no,score,...,month,hour,season,course_type_b9,score_minus_stroke,score_minus_stroke_minus_penalty
0,2021-09-10 12:06:00,1,안성베네스트,West,1,4,...,9,12,4,False,2.0,2.0
1,2021-09-10 12:06:00,1,안성베네스트,West,2,5,...,9,12,4,False,3.0,2.0
2,2021-09-10 12:06:00,1,안성베네스트,West,3,7,...,9,12,4,False,4.0,2.0
3,2021-09-10 12:06:00,1,안성베네스트,West,4,8,...,9,12,4,False,4.0,2.0
4,2021-09-10 12:06:00,1,안성베네스트,West,5,5,...,9,12,4,False,3.0,2.0
5,2021-09-10 12:06:00,1,안성베네스트,West,6,7,...,9,12,4,False,3.0,2.0
6,2021-09-10 12:06:00,1,안성베네스트,West,7,7,...,9,12,4,False,4.0,3.0
7,2021-09-10 12:06:00,1,안성베네스트,West,8,7,...,9,12,4,False,4.0,3.0
8,2021-09-10 12:06:00,1,안성베네스트,West,9,7,...,9,12,4,False,3.0,3.0
9,2021-09-10 12:06:00,1,안성베네스트,North,1,5,...,9,12,4,True,2.0,2.0


  * Prediction DataFrame Information (predict_df_copy):
<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 18 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_time                         612 non-null    datetime64[ns]
 1   no                                612 non-null    int64         
 2   golf_course                       612 non-null    object        
 3   course                            612 non-null    object        
 4   hole_no                           612 non-null    int64         
 5   score                             612 non-null    int64         
 6   stroke                            612 non-null    float64       
 7   putt                              612 non-null    float64       
 8   penalty                           612 non-null    float64       
 9   par                               612 non-null    int64         
 10 

In [ ]:
# 04-4. stroke + penalty + putt vs. score 비교 (240630)
print("04-4. stroke + penalty + putt vs. score 비교")

# stroke + penalty + putt 합계 계산
predict_df_copy['stroke_penalty_putt_sum'] = predict_df_copy['stroke'] + predict_df_copy['penalty'] + predict_df_copy['putt']

# score와의 차이 계산
predict_df_copy['score_difference'] = predict_df_copy['score'] - predict_df_copy['stroke_penalty_putt_sum']

# score_difference가 0인 경우 NaN으로 변환
predict_df_copy['score_difference'] = predict_df_copy['score_difference'].replace(0, np.nan)

# 결과 출력
display(predict_df_copy[['date_time', 'no', 'par', 'stroke', 'penalty', 'putt', 'stroke_penalty_putt_sum', 'score', 'score_difference']].head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

04-4. stroke + penalty + putt vs. score 비교


,date_time,no,par,stroke,penalty,putt,stroke_penalty_putt_sum,score,score_difference
0,2021-09-10 12:06:00,1,4,2.0,0.0,2.0,4.0,4,NaN
1,2021-09-10 12:06:00,1,3,2.0,1.0,2.0,5.0,5,NaN
2,2021-09-10 12:06:00,1,4,3.0,2.0,2.0,7.0,7,NaN
3,2021-09-10 12:06:00,1,5,4.0,2.0,2.0,8.0,8,NaN
4,2021-09-10 12:06:00,1,3,2.0,1.0,2.0,5.0,5,NaN
5,2021-09-10 12:06:00,1,4,4.0,1.0,2.0,7.0,7,NaN
6,2021-09-10 12:06:00,1,4,3.0,1.0,3.0,7.0,7,NaN
7,2021-09-10 12:06:00,1,4,3.0,1.0,3.0,7.0,7,NaN
8,2021-09-10 12:06:00,1,5,4.0,0.0,3.0,7.0,7,NaN
9,2021-09-10 12:06:00,1,5,3.0,0.0,2.0,5.0,5,NaN


  * Prediction DataFrame Information (predict_df_copy):
<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_time                         612 non-null    datetime64[ns]
 1   no                                612 non-null    int64         
 2   golf_course                       612 non-null    object        
 3   course                            612 non-null    object        
 4   hole_no                           612 non-null    int64         
 5   score                             612 non-null    int64         
 6   stroke                            612 non-null    float64       
 7   putt                              612 non-null    float64       
 8   penalty                           612 non-null    float64       
 9   par                               612 non-null    int64         
 10 

In [ ]:
# 04-5. score_difference 조정 및 업데이트 (240630)
print("04-5. score_difference 조정 및 업데이트")

while predict_df_copy['score_difference'].notna().any():
    for index, row in predict_df_copy.iterrows():
        if not pd.isna(row['score_difference']):
            # penalty에 score_difference 추가
            new_penalty = row['penalty'] + row['score_difference']

            if new_penalty < 0:
                # penalty가 음수가 되면 putt에 그 값을 더하고 penalty는 0으로 설정
                predict_df_copy.at[index, 'penalty'] = 0
                predict_df_copy.at[index, 'putt'] += new_penalty
            else:
                # penalty 업데이트
                predict_df_copy.at[index, 'penalty'] = new_penalty

    # stroke, penalty, putt 합계 및 score_difference 재계산
    predict_df_copy['stroke_penalty_putt_sum'] = predict_df_copy['stroke'] + predict_df_copy['penalty'] + predict_df_copy['putt']
    predict_df_copy['score_difference'] = predict_df_copy['score'] - predict_df_copy['stroke_penalty_putt_sum']
    predict_df_copy['score_difference'] = predict_df_copy['score_difference'].replace(0, np.nan)

# 결과 출력
display(predict_df_copy[['date_time', 'no', 'par', 'stroke', 'penalty', 'putt', 'stroke_penalty_putt_sum', 'score', 'score_difference']].head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

print(f"\n  - score_difference 조정을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# ==================================================

04-5. score_difference 조정 및 업데이트


,date_time,no,par,stroke,penalty,putt,stroke_penalty_putt_sum,score,score_difference
0,2021-09-10 12:06:00,1,4,2.0,0.0,2.0,4.0,4,NaN
1,2021-09-10 12:06:00,1,3,2.0,1.0,2.0,5.0,5,NaN
2,2021-09-10 12:06:00,1,4,3.0,2.0,2.0,7.0,7,NaN
3,2021-09-10 12:06:00,1,5,4.0,2.0,2.0,8.0,8,NaN
4,2021-09-10 12:06:00,1,3,2.0,1.0,2.0,5.0,5,NaN
5,2021-09-10 12:06:00,1,4,4.0,1.0,2.0,7.0,7,NaN
6,2021-09-10 12:06:00,1,4,3.0,1.0,3.0,7.0,7,NaN
7,2021-09-10 12:06:00,1,4,3.0,1.0,3.0,7.0,7,NaN
8,2021-09-10 12:06:00,1,5,4.0,0.0,3.0,7.0,7,NaN
9,2021-09-10 12:06:00,1,5,3.0,0.0,2.0,5.0,5,NaN


  * Prediction DataFrame Information (predict_df_copy):
<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_time                         612 non-null    datetime64[ns]
 1   no                                612 non-null    int64         
 2   golf_course                       612 non-null    object        
 3   course                            612 non-null    object        
 4   hole_no                           612 non-null    int64         
 5   score                             612 non-null    int64         
 6   stroke                            612 non-null    float64       
 7   putt                              612 non-null    float64       
 8   penalty                           612 non-null    float64       
 9   par                               612 non-null    int64         
 10 

In [ ]:
# 04-6. fw_hit 예측 및 정수 저장 (240630)
print("04-6. fw_hit 예측 및 정수 저장")

# 파일 경로 설정
output_file_name = '4_fw_hit_predictions.csv'
output_file_path = os.path.join(base_path, sub_directory_name, output_file_name)

# fw_hit 예측을 위한 데이터 준비
X_predict_fw_hit = predict_df_copy[predict_df_copy['par'] != 3][features_fw_hit].dropna()
predicted_fw_hit = model_fw_hit.predict(X_predict_fw_hit)
predict_df_copy.loc[X_predict_fw_hit.index, 'fw_hit'] = np.round(predicted_fw_hit).astype(int)  # 예측값 정수화

print(f"  - 예측을 완료하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터 저장
predict_df_copy.to_csv(output_file_path, index=False, encoding='utf-8-sig')
print(f"  - 데이터프레임을 '{output_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Prediction DataFrame (predict_df_copy):")
display(predict_df_copy.head(18))

# 데이터프레임의 정보 출력
print("  * Prediction DataFrame Information (predict_df_copy):")
display_info_in_sections(predict_df_copy)

# ==================================================

04-6. fw_hit 예측 및 정수 저장
  - 예측을 완료하였습니다. (2024-08-08 16:36:55)
  - 데이터프레임을 '/content/drive/MyDrive/Colab Notebooks/gScore/Output/4_fw_hit_predictions.csv'에 저장하였습니다. (2024-08-08 16:36:55)

  * Prediction DataFrame (predict_df_copy):


,date_time,no,golf_course,course,hole_no,score,...,season,course_type_b9,score_minus_stroke,score_minus_stroke_minus_penalty,stroke_penalty_putt_sum,score_difference
0,2021-09-10 12:06:00,1,안성베네스트,West,1,4,...,4,False,2.0,2.0,4.0,NaN
1,2021-09-10 12:06:00,1,안성베네스트,West,2,5,...,4,False,3.0,2.0,5.0,NaN
2,2021-09-10 12:06:00,1,안성베네스트,West,3,7,...,4,False,4.0,2.0,7.0,NaN
3,2021-09-10 12:06:00,1,안성베네스트,West,4,8,...,4,False,4.0,2.0,8.0,NaN
4,2021-09-10 12:06:00,1,안성베네스트,West,5,5,...,4,False,3.0,2.0,5.0,NaN
5,2021-09-10 12:06:00,1,안성베네스트,West,6,7,...,4,False,3.0,2.0,7.0,NaN
6,2021-09-10 12:06:00,1,안성베네스트,West,7,7,...,4,False,4.0,3.0,7.0,NaN
7,2021-09-10 12:06:00,1,안성베네스트,West,8,7,...,4,False,4.0,3.0,7.0,NaN
8,2021-09-10 12:06:00,1,안성베네스트,West,9,7,...,4,False,3.0,3.0,7.0,NaN
9,2021-09-10 12:06:00,1,안성베네스트,North,1,5,...,4,True,2.0,2.0,5.0,NaN


  * Prediction DataFrame Information (predict_df_copy):
<class 'pandas.core.frame.DataFrame'>
Index: 612 entries, 0 to 611
Data columns (total 20 columns):
 #   Column                            Non-Null Count  Dtype         
---  ------                            --------------  -----         
 0   date_time                         612 non-null    datetime64[ns]
 1   no                                612 non-null    int64         
 2   golf_course                       612 non-null    object        
 3   course                            612 non-null    object        
 4   hole_no                           612 non-null    int64         
 5   score                             612 non-null    int64         
 6   stroke                            612 non-null    float64       
 7   putt                              612 non-null    float64       
 8   penalty                           612 non-null    float64       
 9   par                               612 non-null    int64         
 10 

In [ ]:
# 04-7. 예측된 내용을 최종 파일로 저장 (240630)
print("04-7. 예측된 내용을 최종 파일로 저장")

# 파일 경로 설정
output_csv_file_name = '5_final_predictions.csv'
output_parquet_file_name = '5_final_predictions.parquet'
output_csv_file_path = os.path.join(base_path, sub_directory_name, output_csv_file_name)
output_parquet_file_path = os.path.join(base_path, sub_directory_name, output_parquet_file_name)

# 불필요한 열 제거
columns_to_drop = [
    'is_true', 'month', 'hour', 'season', 'course_type_b9',
    'score_minus_stroke', 'score_minus_stroke_minus_penalty',
    'stroke_penalty_putt_sum', 'score_difference'
]
predict_df_copy.drop(columns=columns_to_drop, inplace=True)

# predict_df_copy 복사하여 predict_temp_df 생성
predict_temp_df = predict_df_copy.copy()

# hole_no_temp 열 추가
predict_temp_df['hole_no_temp'] = predict_temp_df.groupby('no').cumcount() + 1

print("\n  * Final Prediction DataFrame (predict_temp_df):")
display(predict_temp_df)

# 최종 데이터프레임 초기화
final_columns = [
    'no', 'date_time', 'golf_course',
    'stroke_f9_1', 'stroke_f9_2', 'stroke_f9_3', 'stroke_f9_4', 'stroke_f9_5', 'stroke_f9_6', 'stroke_f9_7', 'stroke_f9_8', 'stroke_f9_9',
    'stroke_b9_1', 'stroke_b9_2', 'stroke_b9_3', 'stroke_b9_4', 'stroke_b9_5', 'stroke_b9_6', 'stroke_b9_7', 'stroke_b9_8', 'stroke_b9_9',
    'putt_f9_1', 'putt_f9_2', 'putt_f9_3', 'putt_f9_4', 'putt_f9_5', 'putt_f9_6', 'putt_f9_7', 'putt_f9_8', 'putt_f9_9',
    'putt_b9_1', 'putt_b9_2', 'putt_b9_3', 'putt_b9_4', 'putt_b9_5', 'putt_b9_6', 'putt_b9_7', 'putt_b9_8', 'putt_b9_9',
    'penalty_f9_1', 'penalty_f9_2', 'penalty_f9_3', 'penalty_f9_4', 'penalty_f9_5', 'penalty_f9_6', 'penalty_f9_7', 'penalty_f9_8', 'penalty_f9_9',
    'penalty_b9_1', 'penalty_b9_2', 'penalty_b9_3', 'penalty_b9_4', 'penalty_b9_5', 'penalty_b9_6', 'penalty_b9_7', 'penalty_b9_8', 'penalty_b9_9',
    'fw_hit_f9_1', 'fw_hit_f9_2', 'fw_hit_f9_3', 'fw_hit_f9_4', 'fw_hit_f9_5', 'fw_hit_f9_6', 'fw_hit_f9_7', 'fw_hit_f9_8', 'fw_hit_f9_9',
    'fw_hit_b9_1', 'fw_hit_b9_2', 'fw_hit_b9_3', 'fw_hit_b9_4', 'fw_hit_b9_5', 'fw_hit_b9_6', 'fw_hit_b9_7', 'fw_hit_b9_8', 'fw_hit_b9_9'
]

final_df = pd.DataFrame(columns=final_columns)

for no in predict_temp_df['no'].unique():
    row = {'no': no}
    filtered_df = predict_temp_df[predict_temp_df['no'] == no]

    # 첫번째 행의 date_time과 golf_course 정보 저장
    row['date_time'] = filtered_df.iloc[0]['date_time']
    row['golf_course'] = filtered_df.iloc[0]['golf_course']

    for index, hole in filtered_df.iterrows():
        hole_no_temp = hole['hole_no_temp']
        if hole_no_temp <= 9:
            hole_prefix = 'f9'
            hole_no_str = hole_no_temp
        else:
            hole_prefix = 'b9'
            hole_no_str = hole_no_temp - 9

        row[f'stroke_{hole_prefix}_{hole_no_str}'] = hole['stroke']
        row[f'putt_{hole_prefix}_{hole_no_str}'] = hole['putt']
        row[f'penalty_{hole_prefix}_{hole_no_str}'] = hole['penalty']
        row[f'fw_hit_{hole_prefix}_{hole_no_str}'] = 'Yes' if hole['fw_hit'] == 1 else 'No'

    final_df = pd.concat([final_df, pd.DataFrame([row])], ignore_index=True)

# 데이터 저장 (CSV)
final_df.to_csv(output_csv_file_path, index=False, encoding='utf-8-sig')
print(f"  - 최종 데이터프레임을 '{output_csv_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터 저장 (Parquet)
final_df.to_parquet(output_parquet_file_path, index=False)
print(f"  - 최종 데이터프레임을 '{output_parquet_file_path}'에 저장하였습니다. ({datetime.now(tz).strftime('%Y-%m-%d %H:%M:%S')})")

# 데이터프레임의 데이터 일부 출력
print("\n  * Arranged Final Prediction DataFrame (final_df):")
display(final_df.head(18))

# 데이터프레임의 정보 출력
print("  * Arranged Final Prediction DataFrame Information (final_df):")
display_info_in_sections(final_df)

# ==================================================

04-7. 예측된 내용을 최종 파일로 저장

  * Final Prediction DataFrame (predict_temp_df):


,date_time,no,golf_course,course,hole_no,score,stroke,putt,penalty,par,fw_hit,hole_no_temp
0,2021-09-10 12:06:00,1,안성베네스트,West,1,4,2.0,2.0,0.0,4,0.0,1
1,2021-09-10 12:06:00,1,안성베네스트,West,2,5,2.0,2.0,1.0,3,NaN,2
2,2021-09-10 12:06:00,1,안성베네스트,West,3,7,3.0,2.0,2.0,4,0.0,3
3,2021-09-10 12:06:00,1,안성베네스트,West,4,8,4.0,2.0,2.0,5,0.0,4
4,2021-09-10 12:06:00,1,안성베네스트,West,5,5,2.0,2.0,1.0,3,NaN,5
...,...,...,...,...,...,...,...,...,...,...,...,...
607,2022-09-30 16:09:00,34,강남300,OUT,5,6,3.0,2.0,1.0,3,NaN,14
608,2022-09-30 16:09:00,34,강남300,OUT,6,6,3.0,2.0,1.0,4,0.0,15
609,2022-09-30 16:09:00,34,강남300,OUT,7,6,3.0,2.0,1.0,4,0.0,16
610,2022-09-30 16:09:00,34,강남300,OUT,8,5,3.0,2.0,0.0,4,1.0,17


<ipython-input-58-a1ac0d2782c4>:64: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  final_df = pd.concat([final_df, pd.DataFrame([row])], ignore_index=True)


  - 최종 데이터프레임을 '/content/drive/MyDrive/Colab Notebooks/gScore/Output/5_final_predictions.csv'에 저장하였습니다. (2024-08-08 16:37:05)
  - 최종 데이터프레임을 '/content/drive/MyDrive/Colab Notebooks/gScore/Output/5_final_predictions.parquet'에 저장하였습니다. (2024-08-08 16:37:05)

  * Arranged Final Prediction DataFrame (final_df):


,no,date_time,golf_course,stroke_f9_1,stroke_f9_2,stroke_f9_3,...,fw_hit_b9_4,fw_hit_b9_5,fw_hit_b9_6,fw_hit_b9_7,fw_hit_b9_8,fw_hit_b9_9
0,1,2021-09-10 12:06:00,안성베네스트,2.0,2.0,3.0,...,No,No,No,Yes,Yes,Yes
1,2,2021-09-11 06:59:00,백제,2.0,5.0,4.0,...,Yes,No,No,Yes,Yes,Yes
2,3,2021-10-04 07:51:00,써닝포인트,2.0,2.0,4.0,...,No,No,No,No,Yes,Yes
3,4,2021-10-11 10:25:00,링크나인,2.0,2.0,4.0,...,No,No,Yes,Yes,Yes,No
4,5,2021-10-15 09:00:00,히든밸리,3.0,4.0,3.0,...,Yes,No,No,No,Yes,Yes
5,6,2021-10-29 06:51:00,한림용인,2.0,3.0,5.0,...,No,No,No,Yes,No,Yes
6,7,2021-10-30 08:41:00,이천마이다스,4.0,4.0,2.0,...,No,No,No,Yes,Yes,Yes
7,8,2021-12-08 07:54:00,파크밸리,2.0,4.0,4.0,...,No,No,No,No,No,No
8,9,2022-01-15 08:09:00,강남300,3.0,4.0,3.0,...,No,No,No,No,Yes,Yes
9,10,2022-02-26 12:11:00,레이크사이드,2.0,4.0,3.0,...,No,Yes,No,No,No,Yes


  * Arranged Final Prediction DataFrame Information (final_df):
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34 entries, 0 to 33
Data columns (total 50 columns):
 #   Column        Non-Null Count  Dtype         
---  ------        --------------  -----         
 0   no            34 non-null     object        
 1   date_time     34 non-null     datetime64[ns]
 2   golf_course   34 non-null     object        
 3   stroke_f9_1   34 non-null     float64       
 4   stroke_f9_2   34 non-null     float64       
 5   stroke_f9_3   34 non-null     float64       
 6   stroke_f9_4   34 non-null     float64       
 7   stroke_f9_5   34 non-null     float64       
 8   stroke_f9_6   34 non-null     float64       
 9   stroke_f9_7   34 non-null     float64       
 10  stroke_f9_8   34 non-null     float64       
 11  stroke_f9_9   34 non-null     float64       
 12  stroke_b9_1   34 non-null     float64       
 13  stroke_b9_2   34 non-null     float64       
 14  stroke_b9_3   34 non-null   